In [ ]:
############
# MIT License
#
# Copyright (c) 2026 Rafael Asian, 2023 Minwoo Seong and 2022 MIT CSAIL and Joseph DelPreto
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.
#
# This file is a modified version originally written by Minwoo Seong.
# Source of the original version: https://github.com/dailyminiii/MultiSenseBadminton/blob/main/02_train_networks_example.py
# Date of modification: 2026.01.09
# Modified by: Rafael Asian
# See https://action-net.csail.mit.edu for more usage information.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/TFM_2/utils')

In [ ]:
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 11.3 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import LabelEncoder
from preprocessing import BadmintonDataset
from sklearn.model_selection import KFold
import torch.nn as nn
from SciDataModels import Conv1DRefinedModel, ConvLSTMRefinedModel, LSTMRefinedModel, TransformerRefinedModel, MajorityClassBaseline
from improved_models import (ImprovedConvLSTMModel, EnhancedConvLSTMModel,
                              RestartLearningRateScheduler, SimplifiedConvLSTMModel)
from alternative_models import (
    ResNetLSTMModel,
    InceptionLSTMModel,
    DenseNetLSTMModel,
    TCNModel,
    MobileNetLSTMModel
)
from collections import Counter
from sklearn.metrics import balanced_accuracy_score, f1_score
import os
import argparse
import h5py
import torch
import torch.optim as optim
import random
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import json
from datetime import datetime, timedelta
import pandas as pd
import optuna
from optuna.trial import TrialState
import warnings
import glob
import shutil
import seaborn as sns
from pathlib import Path
from functools import reduce
from itertools import combinations


In [ ]:
# Funciones auxiliares
def plot_confusion_matrix(labels, predictions, task_name, classes, savefolder, normalize=True, cmap=plt.cm.Blues):
    cm = confusion_matrix(labels, predictions)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(f'Confusion matrix for {task_name}')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(savefolder)
    plt.clf()

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

def save(model, config, saved_folder_name, fold=None, epoch=None):
    output_folder_name = saved_folder_name
    if not os.path.exists(output_folder_name):
        os.makedirs(output_folder_name)

    if epoch is None:
        config_name = output_folder_name + '/config_'+ 'S_' + str(fold)
        model_name = output_folder_name + '/model_' + 'S_' + str(fold)
    else:
        config_name = output_folder_name + '/config_'+ 'S_' + str(fold) + '_Ep_' + str(epoch)
        model_name = output_folder_name + '/model_' + 'S_' + str(fold) + '_Ep_' + str(epoch)

    torch.save(model.state_dict(), model_name)
    with open(config_name, 'w') as config_file:
        config_file.write(str(config))

# def leave_three_out_by_skill(subject_ids, skill_levels, seed=42, n_splits=10):
#     random.seed(seed)
#     np.random.seed(seed)

#     unique_subjects = {
#         skill: np.unique([s for s, lvl in zip(subject_ids, skill_levels) if lvl == skill])
#         for skill in np.unique(skill_levels)
#     }

#     combinations = list(itertools.product(*unique_subjects.values()))
#     random.shuffle(combinations)
#     selected_combinations = combinations[:n_splits]

#     splits = []
#     for combo in selected_combinations:
#         test_subjects = [str(s) for s in combo]
#         test_indices = [i for i, sid in enumerate(subject_ids) if sid in test_subjects]
#         train_indices = [i for i in range(len(subject_ids)) if i not in test_indices]
#         splits.append((train_indices, test_indices, test_subjects))

#     return splits

def leave_k_subjects_out(subject_ids, labels=None, n_test_subjects=1, n_val_subjects=1,
                         random_state=42, shuffle=True):
    """
    Genera splits Leave-K-Subjects-Out para cross-validation.

    En cada iteración:
    - Extrae n_test_subjects sujetos para TEST
    - De los restantes, extrae n_val_subjects para VALIDACIÓN
    - El resto va a ENTRENAMIENTO

    Args:
        subject_ids: Array con IDs de sujetos para cada muestra (n_samples,)
        labels: Array con etiquetas (opcional, para verificación) (n_samples,)
        n_test_subjects: Número de sujetos para test en cada fold (default: 1)
        n_val_subjects: Número de sujetos para validación en cada fold (default: 1)
        random_state: Semilla para reproducibilidad
        shuffle: Si True, aleatoriza el orden de los folds

    Yields:
        (train_indices, val_indices, test_indices, val_subjects, test_subjects)

    Ejemplo:
        >>> subject_ids = np.array(['S1','S1','S2','S2','S3','S3','S4','S4'])
        >>> for train, val, test, val_subj, test_subj in leave_k_subjects_out(
        ...     subject_ids, n_test_subjects=1, n_val_subjects=1):
        ...     print(f"Test: {test_subj}, Val: {val_subj}")
    """

    np.random.seed(random_state)
    random.seed(random_state)

    # Obtener sujetos únicos
    unique_subjects = np.unique(subject_ids)
    n_subjects = len(unique_subjects)

    # Validaciones
    if n_test_subjects + n_val_subjects >= n_subjects:
        raise ValueError(
            f"n_test_subjects ({n_test_subjects}) + n_val_subjects ({n_val_subjects}) "
            f"debe ser menor que el número total de sujetos ({n_subjects})"
        )

    if n_test_subjects < 1 or n_val_subjects < 1:
        raise ValueError("n_test_subjects y n_val_subjects deben ser >= 1")

    # print(f"\n{'='*80}")
    # print(f"Leave-K-Subjects-Out Cross-Validation")
    # print(f"{'='*80}")
    # print(f"Total sujetos: {n_subjects}")
    # print(f"Sujetos por fold:")
    # print(f"  - Test: {n_test_subjects}")
    # print(f"  - Validación: {n_val_subjects}")
    # print(f"  - Entrenamiento: {n_subjects - n_test_subjects - n_val_subjects}")
    # print(f"{'='*80}\n")

    # Generar todas las combinaciones posibles de sujetos para test
    test_combinations = list(combinations(unique_subjects, n_test_subjects))

    if shuffle:
        random.shuffle(test_combinations)

    total_folds = len(test_combinations)

    for fold_idx, test_subjects_tuple in enumerate(test_combinations):
        test_subjects = list(test_subjects_tuple)

        # Sujetos restantes después de quitar test
        remaining_subjects = [s for s in unique_subjects if s not in test_subjects]

        # Seleccionar sujetos de validación de los restantes
        val_subjects = random.sample(remaining_subjects, n_val_subjects)

        # Sujetos de entrenamiento
        train_subjects = [s for s in remaining_subjects if s not in val_subjects]

        # Convertir sujetos a índices de muestras
        test_indices = np.where(np.isin(subject_ids, test_subjects))[0]
        val_indices = np.where(np.isin(subject_ids, val_subjects))[0]
        train_indices = np.where(np.isin(subject_ids, train_subjects))[0]

        # Verificar que no hay overlap
        assert len(set(train_indices) & set(val_indices)) == 0, "Train-Val overlap!"
        assert len(set(train_indices) & set(test_indices)) == 0, "Train-Test overlap!"
        assert len(set(val_indices) & set(test_indices)) == 0, "Val-Test overlap!"

        # Información del fold (opcional)
        if labels is not None:
            train_dist = np.bincount(labels[train_indices])
            val_dist = np.bincount(labels[val_indices])
            test_dist = np.bincount(labels[test_indices])
        else:
            train_dist = val_dist = test_dist = None

        yield train_indices, val_indices, test_indices, val_subjects, test_subjects


def get_majority_class(train_dataset):
    labels = [label.item() for _, label in train_dataset]
    most_common_label, _ = Counter(labels).most_common(1)[0]
    return most_common_label


In [ ]:
# Configuración inicial
random_seed = 42
set_random_seed(random_seed)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Suprimir warnings de Optuna
warnings.filterwarnings('ignore', category=UserWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)

DataPath = '/content/drive/MyDrive/TFM_2/data_processed/data_processed_allStreams_60hz_18subj_allActs_skill_level.hdf5'

# Hiperparámetros fijos
PATIENCE_OPTUNA = 5
NUM_EPOCHS_OPTUNA = 30
FOLDS_OPTUNA = 3

PATIENCE_TRAIN = 15
NUM_EPOCHS_TRAIN = 100
# FOLDS_TRAIN = 1

# Configuración de búsqueda
ground_truth_list = ['stroke_type']
model_list = ['LSTM','ConvLSTM','Transformer','ImprovedConvLSTMModel','ResNetLSTMModel']



sensor_subset_list = [
                      'allStreams',
                      # 'noGforce',
                      #  'noCognionics',
                      #  'noEye',
                      #  'noInsole',
                      #  'noBody',
                       'onlyGforce',
                       'onlyCognionics',
                       'onlyEye',
                       'onlyInsole',
                       'onlyBody'
                      ]

technique_list = ['no-technique'
                ]

In [ ]:
# Función objetivo
num_epochs = NUM_EPOCHS_OPTUNA
patience = PATIENCE_OPTUNA

def objective(trial, modelName, technique, ground_truth, subset, base_config):
    """
    Función objetivo para Optuna. Evalúa un modelo con los hiperparámetros dados.

    Early Stopping: Basado en ACCURACY en VALIDACIÓN
    Métrica Optuna: ACCURACY en TEST (promedio de todos los folds)
    """
    # Sugerir hiperparámetros
    batch_size = trial.suggest_categorical('batch_size', [64, 128])
    hidden_features = trial.suggest_categorical('hidden_features', [64, 128])
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True)
    lstm_hidden = trial.suggest_categorical('lstm_hidden', [64, 128])
    dropout_rate = trial.suggest_float('dropout_rate', 0.3, 0.7)
    reduction_ratio = trial.suggest_int('reduction_ratio', 1, 2)
    use_attention = trial.suggest_categorical('use_attention', [True, False])
    num_blocks = trial.suggest_int('num_blocks', 1, 3)
    early_stop_delta = 1e-1

    # Configuración del trial
    config = base_config.copy()
    config['lr'] = learning_rate
    config['batch_size'] = batch_size
    config['hidden_features'] = hidden_features
    config['sensor_subset'] = subset
    config['ground_truth'] = ground_truth
    config['modelName'] = modelName

    # Crear carpeta temporal para este trial
    trial_folder = os.path.join(
        config['base_output_folder'],
        f'trial_{trial.number}'
    )
    config['output_folder_name'] = trial_folder

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Obtener datos
    feature_matrices = h5py.File(DataPath, 'r')['example_matrices'][:]
    feature_matrices_subject_ids = h5py.File(DataPath, 'r')['example_subject_ids'][:]
    feature_matrices_subject_ids_str = np.array([x.decode('utf-8') for x in feature_matrices_subject_ids])
    feature_matrices_stroke_type = h5py.File(DataPath, 'r')['example_label_indexes'][:]
    feature_matrices_skill_level = h5py.File(DataPath, 'r')['example_skill_level'][:]

    feature_matrices_current = feature_matrices
    feature_matrices_ground_truth_current = feature_matrices_stroke_type
    feature_matrices_subject_ids_str_current = feature_matrices_subject_ids_str
    label_num = 3  # Backhand, Forehand, Baseline

    # Aplicar subset de sensores
    if subset == 'allStreams':
        input_feature_matrices = feature_matrices_current
    # elif subset == 'noGforce':
    #     input_feature_matrices = np.concatenate((feature_matrices_current[:, :, :2], feature_matrices_current[:, :, 18:]), axis=2)
    # elif subset == 'noCognionics':
    #     input_feature_matrices = np.concatenate((feature_matrices_current[:, :, :18], feature_matrices_current[:, :, 22:]), axis=2)
    # elif subset == 'noEye':
    #     input_feature_matrices = feature_matrices_current[:, :, 2:]
    # elif subset == 'noInsole':
    #     input_feature_matrices = np.concatenate((feature_matrices_current[:, :, :22], feature_matrices_current[:, :, 58:]), axis=2)
    # elif subset == 'noBody':
    #     input_feature_matrices = feature_matrices_current[:, :, :58]
    elif subset == 'onlyEye': #PIG
        input_feature_matrices = feature_matrices_current[:, :, 0:2]
    elif subset == 'onlyBody': # PNS
        input_feature_matrices = feature_matrices_current[:, :, 58:]
    elif subset == 'onlyGforce':
        input_feature_matrices = feature_matrices_current[:, :, 2:18]
    elif subset == 'onlyCognionics':
        input_feature_matrices = feature_matrices_current[:, :, 18:22]
    elif subset == 'onlyInsole':
        input_feature_matrices = feature_matrices_current[:, :, 22:58]

    # Crear dataset
    dataset = BadmintonDataset(input_feature_matrices, feature_matrices_ground_truth_current)

    # Configuración de splits
    N_TEST_SUBJECTS = 1
    N_VAL_SUBJECTS = 2
    splits = list(leave_k_subjects_out(
        feature_matrices_subject_ids_str_current,
        n_test_subjects=N_TEST_SUBJECTS,
        n_val_subjects=N_VAL_SUBJECTS,
        random_state=42
    ))
    splits = random.sample(splits, FOLDS_OPTUNA)


    # Lista para guardar ACCURACY en TEST de cada fold
    test_accuracies = []

    for fold, (train_indices, val_indices, test_indices, val_subjects, test_subjects) in enumerate(splits):
        # Conjuntos
        train_dataset = torch.utils.data.Subset(dataset, train_indices)
        val_dataset = torch.utils.data.Subset(dataset, val_indices)
        test_dataset = torch.utils.data.Subset(dataset, test_indices)

        # Baseline case
        if modelName == "Baseline":
            majority_class = get_majority_class(train_dataset)
            model = MajorityClassBaseline(majority_class=majority_class, output_size=label_num).to(device)
            model.eval()

            # Test directo
            test_true = [y.item() for _, y in test_dataset]
            test_pred = model(torch.zeros(len(test_dataset), len(input_feature_matrices[0,0,:])).to(device)).cpu().numpy()

            # Calcular ACCURACY en test
            test_acc = np.mean(np.array(test_true) == np.array(test_pred))
            test_accuracies.append(test_acc)
            continue

        # Crear modelo
        if modelName == "ConvLSTM":
            model = ConvLSTMRefinedModel(len(input_feature_matrices[0,0,:]), hidden_features, output_size=label_num).to(device)
        elif modelName == "LSTM":
            model = LSTMRefinedModel(len(input_feature_matrices[0,0,:]), hidden_features, output_size=label_num).to(device)
        elif modelName == "Transformer":
            model = TransformerRefinedModel(len(input_feature_matrices[0,0,:]), hidden_features, output_size=label_num).to(device)
        elif modelName == "ImprovedConvLSTMModel":
            model = ImprovedConvLSTMModel(
                len(input_feature_matrices[0,0,:]),
                hidden_features,
                label_num,
                lstm_hidden,
                reduction_ratio,
                dropout_rate
            ).to(device)
        elif modelName == "ResNetLSTMModel":
            model = ResNetLSTMModel(
                len(input_feature_matrices[0,0,:]),
                hidden_features,
                label_num,
                lstm_hidden,
                num_blocks,
                dropout_rate,
            ).to(device)

        # Entrenar modelo
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # ========== EARLY STOPPING BASADO EN ACCURACY DE VALIDACIÓN ==========
        best_val_accuracy = -float("inf")
        epochs_without_improvement = 0
        min_delta = early_stop_delta
        best_state_dict = None

        for epoch in range(num_epochs):
            # === TRAINING ===
            model.train()
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs.to(device))
                loss = criterion(outputs, targets.long().to(device))
                loss.backward()
                optimizer.step()

            # === VALIDATION ===
            model.eval()
            val_true, val_pred = [], []

            with torch.no_grad():
                for v_inp, v_tar in val_loader:
                    v_out = model(v_inp.to(device))
                    _, v_p = torch.max(v_out, 1)
                    val_true.extend(v_tar.cpu().numpy())
                    val_pred.extend(v_p.cpu().numpy())

            # MÉTRICA:ACCURACY  en validación
            val_accuracy = np.mean(np.array(val_true) == np.array(val_pred))

            # Early stopping basado en accuracy de validación
            if val_accuracy - min_delta > best_val_accuracy:
                best_val_accuracy = val_accuracy
                epochs_without_improvement = 0
                best_state_dict = model.state_dict()
            else:
                epochs_without_improvement += 1

            if epochs_without_improvement >= patience:
                break

            scheduler.step(val_accuracy)

        # === TEST CON MEJOR MODELO ===
        model.load_state_dict(best_state_dict)
        model.eval()

        test_true, test_pred = [], []
        with torch.no_grad():
            for t_inp, t_tar in test_loader:
                t_out = model(t_inp.to(device))
                _, t_p = torch.max(t_out, 1)
                test_true.extend(t_tar.cpu().numpy())
                test_pred.extend(t_p.cpu().numpy())

        # MÉTRICA: ACCURACY en test
        test_accuracy = np.mean(np.array(test_true) == np.array(test_pred))
        test_accuracies.append(test_accuracy)

        # Reportar progreso a Optuna (accuracy promedio en test hasta ahora)
        intermediate_value = np.mean(test_accuracies)
        trial.report(intermediate_value, fold)

        if trial.should_prune():
            raise optuna.TrialPruned()

    # ========== MÉTRICA FINAL PARA OPTUNA ==========
    # Retornar ACCURACY PROMEDIO en TEST de todos los folds
    mean_test_accuracy = np.mean(test_accuracies)

    return mean_test_accuracy

In [ ]:
# Entrenamiento con Optuna
now = datetime.now()
new_time = now + timedelta(hours=1)
date_str = new_time.strftime("%Y-%m-%d")
timestamp = new_time.strftime("%Y-%m-%d_%H-%M-%S")

for modelName in model_list:
    for technique in technique_list:
        for ground_truth in ground_truth_list:
            for subset in sensor_subset_list:

                print(f"\n{'='*80}")
                print(f"Optimizando: {modelName} - {technique} - {ground_truth} - {subset}")
                print(f"{'='*80}\n")

                base_output_folder = os.path.join(
                    '/content/drive/My Drive/TFM_2/resultados_entrenamiento/',
                    date_str,
                    'Fase_1',
                    timestamp,
                    'HPO',
                    f'optuna_{modelName}_{technique}_{ground_truth}_{subset}'
                )

                base_config = {
                    'patience': patience,
                    'max_epochs': num_epochs,
                    'base_output_folder': base_output_folder
                }

                study = optuna.create_study(
                    direction='maximize',
                    # direction='minimize',
                    sampler=optuna.samplers.TPESampler(seed=random_seed),
                    pruner=optuna.pruners.MedianPruner(
                        n_startup_trials=5,
                        n_warmup_steps=1,
                        interval_steps=1
                    )
                )

                n_trials = 10 if modelName != "Baseline" else 1

                study.optimize(
                    lambda trial: objective(trial, modelName, technique, ground_truth, subset, base_config),
                    n_trials=n_trials,
                    timeout=None,
                    show_progress_bar=True
                )


                # Guardar resultados (igual que antes)
                print(f"\n{'='*60}")
                print(f"Mejor trial:")
                print(f"  Métrica de optimización: {study.best_value:.4f}")
                print(f"  Parámetros:")
                for key, value in study.best_params.items():
                    print(f"    {key}: {value}")
                print(f"{'='*60}\n")

                results_dir = os.path.join(base_output_folder)
                os.makedirs(results_dir, exist_ok=True)

                df = study.trials_dataframe()
                df.to_csv(os.path.join(results_dir, 'trials.csv'), index=False)

                best_config = {
                    'best_value': study.best_value,
                    'best_params': study.best_params,
                    'best_trial': study.best_trial.number,
                    'n_trials': len(study.trials),
                    'completed_trials': len([t for t in study.trials if t.state == TrialState.COMPLETE]),
                    'pruned_trials': len([t for t in study.trials if t.state == TrialState.PRUNED])
                }

                with open(os.path.join(results_dir, 'best_config.json'), 'w') as f:
                    json.dump(best_config, f, indent=4)

                if len(study.trials) > 1:
                    try:
                        fig = optuna.visualization.plot_optimization_history(study)
                        fig.write_html(os.path.join(results_dir, 'optimization_history.html'))

                        fig = optuna.visualization.plot_param_importances(study)
                        fig.write_html(os.path.join(results_dir, 'param_importances.html'))

                        print(f" Gráficos guardados en {results_dir}")
                    except Exception as e:
                        print(f"No se pudieron generar algunos gráficos: {e}")
                else:
                    print(f"ℹ No se generaron gráficos (solo 1 trial)")

                print(f"\n Resultados guardados en: {results_dir}\n")


Optimizando: LSTM - no-technique - stroke_type - allStreams



  0%|          | 0/10 [00:00<?, ?it/s]


Mejor trial:
  Métrica de optimización: 0.9890
  Parámetros:
    batch_size: 64
    hidden_features: 64
    learning_rate: 0.0017541893487450805
    lstm_hidden: 64
    dropout_rate: 0.6637281608315128
    reduction_ratio: 1
    use_attention: True
    num_blocks: 2

 Gráficos guardados en /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_LSTM_no-technique_stroke_type_allStreams

 Resultados guardados en: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_LSTM_no-technique_stroke_type_allStreams


Optimizando: LSTM - no-technique - stroke_type - onlyGforce



  0%|          | 0/10 [00:00<?, ?it/s]


Mejor trial:
  Métrica de optimización: 0.6282
  Parámetros:
    batch_size: 64
    hidden_features: 64
    learning_rate: 0.0017541893487450805
    lstm_hidden: 64
    dropout_rate: 0.6637281608315128
    reduction_ratio: 1
    use_attention: True
    num_blocks: 2

 Gráficos guardados en /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_LSTM_no-technique_stroke_type_onlyGforce

 Resultados guardados en: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_LSTM_no-technique_stroke_type_onlyGforce


Optimizando: LSTM - no-technique - stroke_type - onlyCognionics



...


Mejor trial:
  Métrica de optimización: 0.8775
  Parámetros:
    batch_size: 128
    hidden_features: 64
    learning_rate: 0.0020513382630874496
    lstm_hidden: 64
    dropout_rate: 0.6464704583099741
    reduction_ratio: 2
    use_attention: True
    num_blocks: 3

 Gráficos guardados en /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_ResNetLSTMModel_no-technique_stroke_type_onlyEye

 Resultados guardados en: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_ResNetLSTMModel_no-technique_stroke_type_onlyEye


Optimizando: ResNetLSTMModel - no-technique - stroke_type - onlyInsole



  0%|          | 0/10 [00:00<?, ?it/s]


Mejor trial:
  Métrica de optimización: 0.8518
  Parámetros:
    batch_size: 128
    hidden_features: 128
    learning_rate: 0.034877126245459314
    lstm_hidden: 128
    dropout_rate: 0.34634762381005185
    reduction_ratio: 2
    use_attention: True
    num_blocks: 1

 Gráficos guardados en /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_ResNetLSTMModel_no-technique_stroke_type_onlyInsole

 Resultados guardados en: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_ResNetLSTMModel_no-technique_stroke_type_onlyInsole


Optimizando: ResNetLSTMModel - no-technique - stroke_type - onlyBody



  0%|          | 0/10 [00:00<?, ?it/s]


Mejor trial:
  Métrica de optimización: 0.9741
  Parámetros:
    batch_size: 64
    hidden_features: 128
    learning_rate: 0.0040596116104843075
    lstm_hidden: 64
    dropout_rate: 0.41649165607921673
    reduction_ratio: 2
    use_attention: False
    num_blocks: 2

 Gráficos guardados en /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_ResNetLSTMModel_no-technique_stroke_type_onlyBody

 Resultados guardados en: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO/optuna_ResNetLSTMModel_no-technique_stroke_type_onlyBody



In [ ]:
def load_best_params_from_optuna(base_results_folder, modelName, technique, ground_truth, subset):
    """
    Carga los mejores hiperparámetros desde los resultados de Optuna.

    Args:
        base_results_folder: Carpeta base donde están los resultados de Optuna
        modelName: Nombre del modelo
        technique: Técnica (forehand/backhand)
        ground_truth: Variable objetivo
        subset: Subset de sensores

    Returns:
        dict: Diccionario con los mejores hiperparámetros o None si no se encuentra
    """
    # Construir el path esperado
    optuna_folder = os.path.join(
                    base_results_folder,
                    f'optuna_{modelName}_{technique}_{ground_truth}_{subset}'
                )

    config_path = os.path.join(optuna_folder, 'best_config.json')

    if not os.path.exists(config_path):
        print(f"⚠️  No se encontró configuración en: {config_path}")
        return None

    with open(config_path, 'r') as f:
        best_config = json.load(f)

    print(f"✓ Cargada configuración desde Optuna:")
    print(f"  Métrica de optimización: {best_config['best_value']:.4f}")
    print(f"  Parámetros: {best_config['best_params']}")

    return best_config['best_params']

In [ ]:
num_epochs = NUM_EPOCHS_TRAIN
patience = PATIENCE_TRAIN
def train_final_model(modelName, technique, ground_truth, subset, best_params,
                      base_output_folder, splits):
    """
    Entrena el modelo final con los mejores hiperparámetros en todos los folds.

    Args:
        modelName: Nombre del modelo
        technique: Técnica (forehand/backhand)
        ground_truth: Variable objetivo
        subset: Subset de sensores
        best_params: Diccionario con los mejores hiperparámetros
        base_output_folder: Carpeta donde guardar resultados
        splits: Splits para Leave-Three-Out

    Returns:
        dict: Métricas finales del modelo
    """
    print(f"\n{'='*80}")
    print(f"Entrenando modelo final: {modelName} - {technique}")
    print(f"{'='*80}\n")

    # Configuración
    batch_size = best_params['batch_size']
    hidden_features = best_params['hidden_features']
    learning_rate = best_params['learning_rate']
    lstm_hidden = best_params['lstm_hidden']
    dropout_rate = best_params['dropout_rate']
    use_attention = best_params['use_attention']
    reduction_ratio = best_params['reduction_ratio']
    num_blocks = best_params['num_blocks']


    config = {
        'lr': learning_rate,
        'batch_size': batch_size,
        'patience': patience,
        'max_epochs': num_epochs,
        'hidden_features': hidden_features,
        'sensor_subset': subset,
        'ground_truth': ground_truth,
        'modelName': modelName,
        'output_folder_name': base_output_folder,
        'lstm_hidden': lstm_hidden,
        'dropout_rate': dropout_rate,
        'use_attention': use_attention,
        'reduction_ratio': reduction_ratio,
        'num_blocks': num_blocks
    }

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Cargar datos
    feature_matrices = h5py.File(DataPath, 'r')['example_matrices'][:]
    feature_matrices_subject_ids = h5py.File(DataPath, 'r')['example_subject_ids'][:]
    feature_matrices_subject_ids_str = np.array([x.decode('utf-8') for x in feature_matrices_subject_ids])
    feature_matrices_stroke_type = h5py.File(DataPath, 'r')['example_label_indexes'][:]
    feature_matrices_skill_level = h5py.File(DataPath, 'r')['example_skill_level'][:]

    feature_matrices_current = feature_matrices
    feature_matrices_ground_truth_current = feature_matrices_stroke_type
    feature_matrices_subject_ids_str_current = feature_matrices_subject_ids_str
    label_num = 3  # Backhand, Forehand, Baseline

    # Aplicar subset de sensores
    if subset == 'allStreams':
        input_feature_matrices = feature_matrices_current
    # elif subset == 'noGforce':
    #     input_feature_matrices = np.concatenate((feature_matrices_current[:, :, :2], feature_matrices_current[:, :, 18:]), axis=2)
    # elif subset == 'noCognionics':
    #     input_feature_matrices = np.concatenate((feature_matrices_current[:, :, :18], feature_matrices_current[:, :, 22:]), axis=2)
    # elif subset == 'noEye':
    #     input_feature_matrices = feature_matrices_current[:, :, 2:]
    # elif subset == 'noInsole':
    #     input_feature_matrices = np.concatenate((feature_matrices_current[:, :, :22], feature_matrices_current[:, :, 58:]), axis=2)
    # elif subset == 'noBody':
        # input_feature_matrices = feature_matrices_current[:, :, :58]
    elif subset == 'onlyEye':
        input_feature_matrices = feature_matrices_current[:, :, 0:2]
    elif subset == 'onlyBody':
        input_feature_matrices = feature_matrices_current[:, :, 58:]
    elif subset == 'onlyGforce':
        input_feature_matrices = feature_matrices_current[:, :, 2:18]
    elif subset == 'onlyInsole':
        input_feature_matrices = feature_matrices_current[:, :, 22:58]
    elif subset == 'onlyCognionics':
        input_feature_matrices = feature_matrices_current[:, :, 18:22]


    # Crear dataset y splits
    dataset = BadmintonDataset(input_feature_matrices, feature_matrices_ground_truth_current)

    # Guardar información de splits
    os.makedirs(base_output_folder, exist_ok=True)
    with open(os.path.join(base_output_folder, 'leave_three_out_splits.json'), 'w') as f:
        json.dump([list(map(str, combo)) for combo in [s[2] for s in splits]], f, indent=4)

    fold_metrics = []

    print("Labels únicos:", np.unique(feature_matrices_ground_truth_current))
    print("label_num:", label_num)

    for fold, (train_indices, val_indices, test_indices, val_subjects, test_subjects) in enumerate(list(splits)):
      # Conjuntos
      train_dataset = torch.utils.data.Subset(dataset, train_indices)
      val_dataset   = torch.utils.data.Subset(dataset, val_indices)
      test_dataset  = torch.utils.data.Subset(dataset, test_indices)

      print(f"\n{'─'*60}")
      print(f"Fold {fold+1}/{len(splits)} - Test subjects: {test_subjects} - Val subjects: {val_subjects} ")
      print(f"{'─'*60}")

      saved_folder_name = os.path.join(
          base_output_folder,
          f'LTO_{fold+1}_{"_".join(test_subjects)}_{"_".join(val_subjects)}'
      )

        # Caso Baseline
      if modelName == "Baseline":
          majority_class = get_majority_class(train_dataset)
          model = MajorityClassBaseline(majority_class=majority_class, output_size=label_num).to(device)
          model.eval()

          # Test directo (no hay early stopping porque no aprende)
          test_true = [y.item() for _, y in test_dataset]
          test_pred = model(torch.zeros(len(test_dataset), len(input_feature_matrices[0,0,:])).to(device)).cpu().numpy()

          # Metricas de evaluacion
          def eval_baseline(dataset):
              true_labels = [label.item() for _, label in dataset]
              preds = model(torch.zeros(len(dataset), len(input_feature_matrices[0,0,:])).to(device)).cpu().numpy()
              acc = np.mean(np.array(true_labels) == np.array(preds))
              bal_acc = balanced_accuracy_score(true_labels, preds)
              f1 = f1_score(true_labels, preds, average='weighted')
              return true_labels, preds, acc, bal_acc, f1

          train_true_labels, train_preds, train_acc, bal_acc_train, f1_train = eval_baseline(train_dataset)
          test_true_labels, test_preds, test_acc, bal_acc_test, f1_test = eval_baseline(test_dataset)

          fold_metrics.append({
              "fold": fold + 1,
              "test_subjects": "_".join(test_subjects),
              "val_subjects": "_".join(val_subjects),
              "train_loss": 0.0,
              "train_acc": train_acc,
              "test_loss": 0.0,
              "test_acc": test_acc,
              "train_bal_acc": bal_acc_train,
              "test_bal_acc": bal_acc_test,
              "train_f1_acc": f1_train,
              "test_f1_acc": f1_test
          })

          print(f" Train Acc: {train_acc:.4f} |  Test Acc: {test_acc:.4f} | Bal Acc: {bal_acc_test:.4f} | F1: {f1_test:.4f}")
          continue

      # Crear carpeta para este fold
      os.makedirs(saved_folder_name, exist_ok=True)

      train_dataset = torch.utils.data.Subset(dataset, train_indices)
      test_dataset = torch.utils.data.Subset(dataset, test_indices)

      # Crear modelo
      if modelName == "Conv1D":
          model = Conv1DRefinedModel(len(input_feature_matrices[0,0,:]), hidden_features, output_size=label_num).to(device)
      elif modelName == "LSTM":
          model = LSTMRefinedModel(len(input_feature_matrices[0,0,:]), hidden_features, output_size=label_num).to(device)
      elif modelName == "ConvLSTM":
          model = ConvLSTMRefinedModel(len(input_feature_matrices[0,0,:]), hidden_features, output_size=label_num).to(device)
      elif modelName == "Transformer":
          model = TransformerRefinedModel(len(input_feature_matrices[0,0,:]), hidden_features, output_size=label_num).to(device)
      elif modelName == "ImprovedConvLSTMModel":
          model = ImprovedConvLSTMModel(
              len(input_feature_matrices[0,0,:]),
              hidden_features,
              label_num,
              lstm_hidden,
              reduction_ratio,
              dropout_rate
              ).to(device)
      elif modelName == "ResNetLSTMModel":
          model = ResNetLSTMModel(
              len(input_feature_matrices[0,0,:]),
              hidden_features,
              label_num,
              lstm_hidden,
              num_blocks,
              dropout_rate,
              ).to(device)

      # Entrenamiento del modelo
      criterion = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
      scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.3, patience=5)

      train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
      val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
      test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


      # Listas para tracking
      train_losses, train_accs = [], []
      test_losses, test_accs = [], []
      balanced_acc_train_list, f1_weighted_train_list = [], []
      balanced_acc_test_list, f1_weighted_test_list = [], []

      best_val_metric = -float("inf")
      epochs_without_improvement = 0
      min_delta = 1e-2

      # Para cargar el mejor modelo luego
      best_state_dict = None

      for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        train_true_labels, train_preds = [], []

        for inputs, targets in train_loader:
          optimizer.zero_grad()
          outputs = model(inputs.to(device))
          loss = criterion(outputs, targets.long().to(device))
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
          _, predicted = torch.max(outputs, 1)
          total += targets.size(0)
          correct += (predicted == targets.long().to(device)).sum().item()

          train_true_labels.extend(targets.cpu().numpy())
          train_preds.extend(predicted.cpu().detach().numpy())
          bal_acc_train = balanced_accuracy_score(train_true_labels, train_preds)
          train_acc = correct / total

        # Validation
        model.eval()
        val_true, val_pred = [], []

        with torch.no_grad():
            for v_inp, v_tar in val_loader:
                v_out = model(v_inp.to(device))
                _, v_p = torch.max(v_out, 1)
                val_true.extend(v_tar.cpu().numpy())
                val_pred.extend(v_p.cpu().numpy())

        val_metric = balanced_accuracy_score(val_true, val_pred)

        # Early stopping logic
        if val_metric - min_delta > best_val_metric:
            best_val_metric = val_metric
            epochs_without_improvement = 0
            best_state_dict = model.state_dict()  # <--- Guardamos mejor modelo
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            break

        scheduler.step(val_metric)
        print(f"Epoca {epoch} | Bal Val Acc: {val_metric:.4f} |  Train Acc: {train_acc:.4f} |  Bal Train Acc: {bal_acc_train:.4f} | Loss: {running_loss / len(train_loader):.4f} | Epocas sin mejorar: {epochs_without_improvement}")

      # Test
      model.load_state_dict(best_state_dict)
      model.eval()

      correct_test = 0
      total_test = 0
      test_true_labels = []
      test_preds = []
      test_loss_total = 0.0

      with torch.no_grad():
          for test_inputs, test_targets in test_loader:
              test_outputs = model(test_inputs.to(device))
              test_loss = criterion(test_outputs, test_targets.long().to(device))
              test_loss_total += test_loss.item()

              _, predicted = torch.max(test_outputs, 1)
              total_test += test_targets.size(0)
              correct_test += (predicted == test_targets.long().to(device)).sum().item()

              test_true_labels.extend(test_targets.cpu().numpy())
              test_preds.extend(predicted.cpu().numpy())

      test_acc = correct_test / total_test
      bal_acc_test = balanced_accuracy_score(test_true_labels, test_preds)
      f1_test = f1_score(test_true_labels, test_preds, average='weighted')
      test_losses.append(test_loss_total / len(test_loader))

      # Calcular métricas
      train_acc = correct / total
      bal_acc_train = balanced_accuracy_score(train_true_labels, train_preds)
      f1_train = f1_score(train_true_labels, train_preds, average='weighted')

      train_losses.append(running_loss / len(train_loader))
      train_accs.append(train_acc)

      test_accs.append(test_acc)
      balanced_acc_train_list.append(bal_acc_train)
      f1_weighted_train_list.append(f1_train)
      balanced_acc_test_list.append(bal_acc_test)
      f1_weighted_test_list.append(f1_test)

      # Guardar gráficos
      plt.figure(figsize=(8, 8))
      plt.plot(train_losses, label='Train Loss')
      plt.plot(test_losses, label='Test Loss')
      plt.xlabel('Epochs')
      plt.ylabel('Loss')
      plt.legend()
      plt.savefig(os.path.join(saved_folder_name, 'loss.png'))
      plt.clf()

      plt.figure(figsize=(8, 8))
      plt.plot(train_accs, label='Train Accuracy')
      plt.plot(test_accs, label='Test Accuracy')
      plt.xlabel('Epochs')
      plt.ylabel('Accuracy')
      plt.legend()
      plt.savefig(os.path.join(saved_folder_name, 'accuracy.png'))
      plt.clf()

      plt.figure(figsize=(8, 8))
      plt.plot(balanced_acc_train_list, label='Train Balanced Accuracy')
      plt.plot(balanced_acc_test_list, label='Test Balanced Accuracy')
      plt.xlabel('Epochs')
      plt.ylabel('Balanced Accuracy')
      plt.legend()
      plt.savefig(os.path.join(saved_folder_name, 'bal_acc.png'))
      plt.clf()

      plt.figure(figsize=(8, 8))
      plt.plot(f1_weighted_train_list, label='Train F1')
      plt.plot(f1_weighted_test_list, label='Test F1')
      plt.xlabel('Epochs')
      plt.ylabel('F1 Score')
      plt.legend()
      plt.savefig(os.path.join(saved_folder_name, 'f1_acc.png'))
      plt.clf()

      # Matrices de confusión
      plot_confusion_matrix(train_true_labels, train_preds, "skill_level", [0,1,2],
                          os.path.join(saved_folder_name, 'train_acc_confusion.png'))
      plot_confusion_matrix(test_true_labels, test_preds, "skill_level", [0,1,2],
                          os.path.join(saved_folder_name, 'test_acc_confusion.png'))

      # Guardar modelo y configuración
      config['train_loss'] = train_losses[-1]
      config['train_acc'] = train_accs[-1]
      config['test_loss'] = test_losses[-1]
      config['test_acc'] = test_accs[-1]
      config['train_bal_acc'] = bal_acc_train
      config['test_bal_acc'] = bal_acc_test
      config['train_f1_acc'] = f1_train
      config['test_f1_acc'] = f1_test

      save(model, config, saved_folder_name, fold=fold+1, epoch=epoch+1)

      fold_metrics.append({
          "fold": fold + 1,
          "test_subjects": "_".join(test_subjects),
          "val_subjects": "_".join(val_subjects),
          "train_loss": config['train_loss'],
          "train_acc": config['train_acc'],
          "test_loss": config['test_loss'],
          "test_acc": config['test_acc'],
          "train_bal_acc": config['train_bal_acc'],
          "test_bal_acc": config['test_bal_acc'],
          "train_f1_acc": config['train_f1_acc'],
          "test_f1_acc": config['test_f1_acc']
      })

      print(f"Train Acc:{train_acc} | Test Acc: {test_acc:.4f} | Bal Test Acc: {bal_acc_test:.4f} | F1 Test: {f1_test:.4f}")

    # Guardar resumen de todos los folds
    folds_df = pd.DataFrame(fold_metrics)

    summary = {
        "metric": ["train_acc","test_acc", "train_bal_acc","test_bal_acc", "train_f1_acc","test_f1_acc", "train_loss","test_loss"],
        "mean": [
            folds_df["train_acc"].mean() * 100,
            folds_df["test_acc"].mean() * 100,
            folds_df["train_bal_acc"].mean() * 100,
            folds_df["test_bal_acc"].mean() * 100,
            folds_df["train_f1_acc"].mean() * 100,
            folds_df["test_f1_acc"].mean() * 100,
            folds_df["train_loss"].mean(),
            folds_df["test_loss"].mean()
        ],
        "std": [
            folds_df["train_acc"].std() * 100,
            folds_df["test_acc"].std() * 100,
            folds_df["train_bal_acc"].std() * 100,
            folds_df["test_bal_acc"].std() * 100,
            folds_df["train_f1_acc"].std() * 100,
            folds_df["test_f1_acc"].std() * 100,
            folds_df["train_loss"].std(),
            folds_df["test_loss"].std()
        ]
    }

    summary_df = pd.DataFrame(summary)
    results_dir = os.path.join(base_output_folder, "fold_metrics")
    os.makedirs(results_dir, exist_ok=True)

    folds_csv_path = os.path.join(results_dir, f"{modelName}_folds_results.csv")
    folds_df.to_csv(folds_csv_path, index=False)

    summary_csv_path = os.path.join(results_dir, f"{modelName}_summary.csv")
    summary_df.to_csv(summary_csv_path, index=False)

    summary_json_path = os.path.join(results_dir, f"{modelName}_summary.json")
    with open(summary_json_path, "w") as f:
        json.dump(summary_df.to_dict(orient="records"), f, indent=4)

    print(f"\n{'='*80}")
    print(f"Resumen Final - {modelName} ({technique}):")
    print(f"  Train Accuracy:    {folds_df['train_acc'].mean()*100:.2f}% ± {folds_df['train_acc'].std()*100:.2f}%")
    print(f"  Test Accuracy:     {folds_df['test_acc'].mean()*100:.2f}% ± {folds_df['test_acc'].std()*100:.2f}%")
    print(f"  Test Bal Accuracy: {folds_df['test_bal_acc'].mean()*100:.2f}% ± {folds_df['test_bal_acc'].std()*100:.2f}%")
    print(f"  Test F1:           {folds_df['test_f1_acc'].mean()*100:.2f}% ± {folds_df['test_f1_acc'].std()*100:.2f}%")
    print(f"{'='*80}\n")

    return fold_metrics

In [ ]:
# timestamp =  '2025-12-08_00-27-50'
# date_str = '2025-12-08'


#
OPTUNA_BASE_FOLDER = os.path.join(
                    '/content/drive/My Drive/TFM_2/resultados_entrenamiento/',
                    date_str,
                    'Fase_1',
                    timestamp,
                    'HPO'
                )

# Crear nueva carpeta para resultados finales
FINAL_BASE_FOLDER = os.path.join(
                    '/content/drive/My Drive/TFM_2/resultados_entrenamiento/',
                    date_str,
                    'Fase_1',
                    timestamp,
                    'resultados_entrenamiento'
                )

# Generamos los splits

feature_matrices = h5py.File(DataPath, 'r')['example_matrices'][:]
feature_matrices_subject_ids = h5py.File(DataPath, 'r')['example_subject_ids'][:]
feature_matrices_subject_ids_str = np.array([x.decode('utf-8') for x in feature_matrices_subject_ids])

N_TEST_SUBJECTS = 1
N_VAL_SUBJECTS = 2
splits = list(leave_k_subjects_out(
    feature_matrices_subject_ids_str,
    n_test_subjects=N_TEST_SUBJECTS,
    n_val_subjects=N_VAL_SUBJECTS,
    random_state=42
))
# Creamos los scripts de entrenamiento

print(f"\n{'#'*80}")
print(f"# ENTRENAMIENTO FINAL CON MEJORES HIPERPARÁMETROS DE OPTUNA")
print(f"# Leyendo desde: {OPTUNA_BASE_FOLDER}")
print(f"# Guardando en: {FINAL_BASE_FOLDER}")
print(f"{'#'*80}\n")

# Entrenar cada combinación
for modelName in model_list:
    for technique in technique_list:
        for ground_truth in ground_truth_list:
            for subset in sensor_subset_list:
                print(f"\n{'='*80}")
                print(f"Procesando: {modelName} - {technique} - {ground_truth} - {subset}")
                print(f"{'='*80}")

                # Cargar mejores hiperparámetros de Optuna
                best_params = load_best_params_from_optuna(
                    OPTUNA_BASE_FOLDER,
                    modelName,
                    technique,
                    ground_truth,
                    subset
                )

                if best_params is None:
                    print(f"Saltando {modelName}-{technique} (no hay configuración)")
                    continue

                # Crear carpeta de salida
                output_folder = os.path.join(
                    FINAL_BASE_FOLDER,
                    f'{modelName}_{technique}_{ground_truth}_{subset}'
                )

                # Entrenar modelo final con todos los folds
                try:
                    fold_metrics = train_final_model(
                        modelName=modelName,
                        technique=technique,
                        ground_truth=ground_truth,
                        subset=subset,
                        best_params=best_params,
                        base_output_folder=output_folder,
                        splits = splits  # Usar todos los splits para el modelo final
                    )

                    print(f"Completado exitosamente")

                except Exception as e:
                    print(f"Error durante el entrenamiento: {str(e)}")
                    import traceback
                    traceback.print_exc()

print(f"\n{'#'*80}")
print(f"# ENTRENAMIENTO FINAL COMPLETADO")
print(f"# Resultados guardados en: {FINAL_BASE_FOLDER}")
print(f"{'#'*80}\n")


################################################################################
# ENTRENAMIENTO FINAL CON MEJORES HIPERPARÁMETROS DE OPTUNA
# Leyendo desde: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/HPO
# Guardando en: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-13/Fase_1/2025-12-13_00-16-40/resultados_entrenamiento
################################################################################


Procesando: LSTM - no-technique - stroke_type - onlyBody
✓ Cargada configuración desde Optuna:
  Métrica de optimización: 0.9845
  Parámetros: {'batch_size': 128, 'hidden_features': 128, 'learning_rate': 0.015304852121831466, 'lstm_hidden': 128, 'dropout_rate': 0.3682096494749166, 'reduction_ratio': 1, 'use_attention': False, 'num_blocks': 3}

Entrenando modelo final: LSTM - no-technique

Labels únicos: [0 1 2]
label_num: 3

────────────────────────────────────────────────────────────
Fold 1/18 - Test subjects: [np.str_('

/tmp/ipython-input-2429421797.py:327: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 8))


Train Acc:1.0 | Test Acc: 0.9884 | Bal Test Acc: 0.9867 | F1 Test: 0.9884

────────────────────────────────────────────────────────────
Fold 5/18 - Test subjects: [np.str_('Sub07')] - Val subjects: [np.str_('Sub18'), np.str_('Sub15')] 
────────────────────────────────────────────────────────────
Epoca 0 | Bal Val Acc: 0.9410 |  Train Acc: 0.6957 |  Bal Train Acc: 0.6748 | Loss: 2.9128 | Epocas sin mejorar: 0
Epoca 1 | Bal Val Acc: 0.9681 |  Train Acc: 0.9538 |  Bal Train Acc: 0.9517 | Loss: 0.1317 | Epocas sin mejorar: 0
Epoca 2 | Bal Val Acc: 0.9592 |  Train Acc: 0.9783 |  Bal Train Acc: 0.9765 | Loss: 0.0633 | Epocas sin mejorar: 1
Epoca 3 | Bal Val Acc: 0.9807 |  Train Acc: 0.9860 |  Bal Train Acc: 0.9854 | Loss: 0.0452 | Epocas sin mejorar: 0
Epoca 4 | Bal Val Acc: 0.9807 |  Train Acc: 0.9908 |  Bal Train Acc: 0.9907 | Loss: 0.0318 | Epocas sin mejorar: 1
Epoca 5 | Bal Val Acc: 0.9719 |  Train Acc: 0.9886 |  Bal Train Acc: 0.9879 | Loss: 0.0322 | Epocas sin mejorar: 2
Epoca 6 | Bal

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

...

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [ ]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def create_metrics_heatmap_per_model(results_dict, output_folder, dpi=300):
    """
    Crea heatmaps separados para cada modelo: uno de métricas de precisión y otro de pérdida.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar los heatmaps
        dpi: Resolución de las imágenes
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Diccionario de traducción de métricas
    metric_translation = {
        'test_acc': 'Exactitud Test',
        'test_bal_acc': 'Exactitud Bal. Test',
        'test_f1_acc': 'F1 Test',
        'train_acc': 'Exactitud Entren.',
        'train_bal_acc': 'Exactitud Bal. Entren.',
        'train_f1_acc': 'F1 Entren.',
        'test_loss': 'Pérdida Test',
        'train_loss': 'Pérdida Entren.'
    }

    for model_name, df in results_dict.items():

        # Preparar datos para el heatmap
        heatmap_data = []
        metrics_columns = []

        # Extraer valores numéricos de cada métrica
        for col in df.columns:
            if col != 'sensor_subset':
                # Extraer solo el valor medio (antes del ±)
                values = df[col].apply(lambda x: float(str(x).split('±')[0].strip()))
                heatmap_data.append(values.values)
                metrics_columns.append(metric_translation.get(col, col))

        # Crear DataFrame para heatmap (transpuesto)
        sensor_subsets = df['sensor_subset'].values
        heatmap_df = pd.DataFrame(
            np.array(heatmap_data).T,
            index=sensor_subsets,
            columns=metrics_columns
        )

        # Separar métricas de accuracy/F1 de métricas de loss
        accuracy_metrics = [col for col in heatmap_df.columns if 'Pérdida' not in col]
        loss_metrics = [col for col in heatmap_df.columns if 'Pérdida' in col]

        # ========== HEATMAP 1: MÉTRICAS DE PRECISIÓN ==========
        if accuracy_metrics:
            fig, ax = plt.subplots(figsize=(12, max(6, len(sensor_subsets) * 0.5)))

            # Usar colormap más intenso
            sns.heatmap(heatmap_df[accuracy_metrics],
                       annot=True,
                       fmt='.2f',
                       cmap='RdYlGn',
                       center=None,  # Sin centro para mayor intensidad
                       linewidths=0.8,
                       linecolor='white',
                       cbar_kws={'label': 'Valor (%)'},
                       ax=ax,
                       vmin=heatmap_df[accuracy_metrics].values.min() - 5,  # Rango más ajustado
                       vmax=100,
                       annot_kws={'fontsize': 9, 'weight': 'bold'})

            ax.set_title(f'{model_name} - Métricas de Rendimiento',
                         fontsize=14, fontweight='bold', pad=15)
            ax.set_xlabel('Métrica', fontsize=12, fontweight='bold')
            ax.set_ylabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontweight='bold')
            ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontweight='bold')

            plt.tight_layout()

            # Guardar heatmap de precisión
            output_path = os.path.join(output_folder, f'{model_name}_heatmap_precision.png')
            plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
            plt.close()

            print(f"Heatmap de precisión generado: {output_path}")

        # ========== HEATMAP 2: MÉTRICAS DE PÉRDIDA ==========
        if loss_metrics:
            fig, ax = plt.subplots(figsize=(8, max(6, len(sensor_subsets) * 0.5)))

            # Usar colormap más intenso para pérdida
            sns.heatmap(heatmap_df[loss_metrics],
                       annot=True,
                       fmt='.4f',
                       cmap='RdYlGn_r',
                       center=None,  # Sin centro para mayor intensidad
                       linewidths=0.8,
                       linecolor='white',
                       cbar_kws={'label': 'Valor'},
                       ax=ax,
                       annot_kws={'fontsize': 9, 'weight': 'bold'})

            ax.set_title(f'{model_name} - Métricas de Pérdida',
                         fontsize=14, fontweight='bold', pad=15)
            ax.set_xlabel('Métrica', fontsize=12, fontweight='bold')
            ax.set_ylabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontweight='bold')
            ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontweight='bold')

            plt.tight_layout()

            # Guardar heatmap de pérdida
            output_path = os.path.join(output_folder, f'{model_name}_heatmap_perdida.png')
            plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
            plt.close()

            print(f"Heatmap de pérdida generado: {output_path}")


def create_single_combined_heatmap(results_dict, output_folder, dpi=300):
    """
    Crea un único heatmap combinado mostrando todas las métricas de test
    para todos los modelos y subconjuntos de sensores.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar el heatmap
        dpi: Resolución de la imagen
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Métricas de test a incluir
    test_metrics = ['test_acc', 'test_bal_acc', 'test_f1_acc']

    # Diccionario de traducción
    metric_translation = {
        'test_acc': 'Exactitud',
        'test_bal_acc': 'Exactitud Bal.',
        'test_f1_acc': 'F1 Score'
    }

    # Preparar datos combinados
    combined_data = []
    index_labels = []

    for model_name, df in results_dict.items():
        for _, row in df.iterrows():
            subset = row['sensor_subset']
            label = f"{model_name} - {subset}"
            index_labels.append(label)

            row_data = []
            for metric in test_metrics:
                if metric in df.columns:
                    value = float(str(row[metric]).split('±')[0].strip())
                    row_data.append(value)
                else:
                    row_data.append(np.nan)

            combined_data.append(row_data)

    # Crear DataFrame
    combined_df = pd.DataFrame(
        combined_data,
        index=index_labels,
        columns=[metric_translation.get(m, m) for m in test_metrics]
    )

    # Crear figura
    fig_height = max(12, len(index_labels) * 0.4)
    fig, ax = plt.subplots(figsize=(10, fig_height))

    # Crear heatmap con colores más intensos
    sns.heatmap(combined_df,
               annot=True,
               fmt='.2f',
               cmap='RdYlGn',
               center=None,  # Sin centro para mayor intensidad
               linewidths=0.8,
               linecolor='white',
               cbar_kws={'label': 'Valor (%)'},
               ax=ax,
               vmin=combined_df.values.min() - 5,
               vmax=100,
               annot_kws={'fontsize': 8, 'weight': 'bold'})

    ax.set_title('Comparación General de Métricas de Test - Todos los Modelos',
                fontsize=14, fontweight='bold', pad=15)
    ax.set_xlabel('Métrica', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo - Subconjunto de Sensores', fontsize=12, fontweight='bold')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontweight='bold')
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=8, fontweight='bold')

    plt.tight_layout()

    # Guardar
    output_path = os.path.join(output_folder, 'heatmap_general_todos_modelos.png')
    plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"Heatmap general combinado generado: {output_path}")


def create_metrics_comparison_per_subset(results_dict, output_folder, dpi=300):
    """
    Crea heatmaps mostrando cómo cada subconjunto de sensores se comporta
    en diferentes modelos para cada métrica.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar los heatmaps
        dpi: Resolución de la imagen
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Obtener todos los subconjuntos de sensores
    all_subsets = results_dict[list(results_dict.keys())[0]]['sensor_subset'].values

    # Métricas a visualizar
    metrics_to_plot = {
        'test_acc': 'Exactitud Test (%)',
        'test_bal_acc': 'Exactitud Balanceada Test (%)',
        'test_f1_acc': 'F1 Score Test (%)'
    }

    for metric, metric_label in metrics_to_plot.items():

        # Preparar datos para este métrica
        heatmap_data = []
        model_names = []

        for model_name, df in results_dict.items():
            if metric in df.columns:
                values = df[metric].apply(lambda x: float(str(x).split('±')[0].strip()))
                heatmap_data.append(values.values)
                model_names.append(model_name)

        if not heatmap_data:
            continue

        # Crear DataFrame
        heatmap_df = pd.DataFrame(
            np.array(heatmap_data),
            index=model_names,
            columns=all_subsets
        )

        # Crear figura
        fig, ax = plt.subplots(figsize=(14, max(6, len(model_names) * 0.8)))

        # Crear heatmap con colores más intensos
        sns.heatmap(heatmap_df,
                   annot=True,
                   fmt='.2f',
                   cmap='RdYlGn',
                   center=None,  # Sin centro para mayor intensidad
                   linewidths=0.8,
                   linecolor='white',
                   cbar_kws={'label': metric_label},
                   ax=ax,
                   vmin=heatmap_df.values.min() - 5,
                   vmax=100,
                   annot_kws={'fontsize': 9, 'weight': 'bold'})

        ax.set_title(f'Comparación de {metric_label} por Modelo y Subconjunto',
                    fontsize=14, fontweight='bold', pad=15)
        ax.set_xlabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
        ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontweight='bold')
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontweight='bold')

        plt.tight_layout()

        # Guardar
        output_path = os.path.join(output_folder, f'heatmap_por_subset_{metric}.png')
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
        plt.close()

        print(f"Heatmap por subset generado: {output_path}")


# ============================================================================
# EJECUCIÓN - HEATMAPS DE MÉTRICAS
# ============================================================================

timestamp = '2025-12-08_00-27-50'
date_str = '2025-12-08'

FINAL_BASE_FOLDER = os.path.join(
    '/content/drive/My Drive/TFM_2/resultados_entrenamiento/',
    date_str,
    'Fase_1',
    timestamp,
    'resultados_entrenamiento'
)

results = extract_and_compare_results(
    base_folder=FINAL_BASE_FOLDER,
    model_list=model_list,
    sensor_subset_list=sensor_subset_list,
    ground_truth='stroke_type',
    technique='no-technique'
)

# Crear carpeta específica para heatmaps
heatmap_output = os.path.join(FINAL_BASE_FOLDER, 'heatmaps_metricas')

print("\n" + "="*80)
print("GENERANDO HEATMAPS DE MÉTRICAS")
print("="*80 + "\n")

# Heatmaps separados por cada modelo (precisión y pérdida en archivos distintos)
create_metrics_heatmap_per_model(results, heatmap_output, dpi=300)

# Heatmap general combinado (todos los modelos y subsets)
create_single_combined_heatmap(results, heatmap_output, dpi=300)

# Heatmaps por métrica específica (modelos vs subsets)
create_metrics_comparison_per_subset(results, heatmap_output, dpi=300)

print("\n" + "#"*80)
print("# HEATMAPS DE MÉTRICAS COMPLETADOS")
print(f"# Archivos guardados en: {heatmap_output}")
print("#"*80 + "\n")


Resultados para modelo: LSTM
 sensor_subset      test_acc  test_bal_acc   test_f1_acc     train_acc train_bal_acc  train_f1_acc       test_loss      train_loss
    allStreams 90.85 ± 14.48 88.78 ± 16.70 90.40 ± 15.17 100.00 ± 0.02 100.00 ± 0.02 100.00 ± 0.02 0.4645 ± 1.0184 0.0003 ± 0.0005
    onlyGforce 54.13 ± 22.36 49.59 ± 21.05 51.86 ± 23.13  86.80 ± 3.24  84.61 ± 3.94  86.65 ± 3.32 2.4161 ± 2.5970 0.3173 ± 0.0726
onlyCognionics 51.06 ± 12.45 40.77 ± 13.16 43.87 ± 14.07  71.44 ± 4.12  67.35 ± 4.77  70.83 ± 4.35 2.1276 ± 2.1345 0.6054 ± 0.0681
       onlyEye 75.95 ± 20.32 74.04 ± 20.81 75.36 ± 20.68  92.00 ± 2.99  91.09 ± 3.34  91.99 ± 3.00 0.9877 ± 1.4008 0.2072 ± 0.0722
    onlyInsole 81.44 ± 14.10 77.50 ± 16.09 79.70 ± 15.35  97.41 ± 1.37  97.00 ± 1.62  97.41 ± 1.38 0.7572 ± 0.7920 0.0772 ± 0.0382
      onlyBody 89.69 ± 16.23 88.24 ± 18.11 89.16 ± 17.11 100.00 ± 0.00 100.00 ± 0.01 100.00 ± 0.00 0.5974 ± 1.1677 0.0011 ± 0.0003


Resultados para modelo: ConvLSTM
 sensor_subset    

In [ ]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

def extract_and_compare_results(base_folder, model_list, sensor_subset_list,
                                ground_truth='stroke_type', technique='no-technique'):
    """
    Extrae y compara resultados de entrenamiento para diferentes modelos y subconjuntos de sensores.

    Args:
        base_folder: Ruta base donde se encuentran los resultados
        model_list: Lista de nombres de modelos a comparar
        sensor_subset_list: Lista de subconjuntos de sensores utilizados
        ground_truth: Variable objetivo utilizada
        technique: Técnica utilizada en el entrenamiento

    Returns:
        dict: Diccionario con DataFrames de comparación por modelo
    """

    results_by_model = {}

    for model_name in model_list:
        comparison_data = []

        for subset in sensor_subset_list:
            # Construir ruta al archivo de resumen
            config_folder = f'{model_name}_{technique}_{ground_truth}_{subset}'
            summary_path = os.path.join(
                base_folder,
                config_folder,
                'fold_metrics',
                f'{model_name}_summary.csv'
            )

            # Verificar si existe el archivo
            if not os.path.exists(summary_path):
                print(f"Advertencia: No se encontró {summary_path}")
                continue

            # Leer resumen
            summary_df = pd.read_csv(summary_path)

            # Extraer métricas relevantes
            metrics_dict = {'sensor_subset': subset}

            for _, row in summary_df.iterrows():
                metric_name = row['metric']
                mean_val = row['mean']
                std_val = row['std']

                # Formato: métrica ± desviación
                if 'loss' in metric_name:
                    metrics_dict[metric_name] = f"{mean_val:.4f} ± {std_val:.4f}"
                else:
                    metrics_dict[metric_name] = f"{mean_val:.2f} ± {std_val:.2f}"

            comparison_data.append(metrics_dict)

        # Crear DataFrame de comparación
        if comparison_data:
            comparison_df = pd.DataFrame(comparison_data)

            # Reordenar columnas para mejor visualización
            cols_order = ['sensor_subset', 'test_acc', 'test_bal_acc', 'test_f1_acc',
                         'train_acc', 'train_bal_acc', 'train_f1_acc',
                         'test_loss', 'train_loss']

            # Mantener solo columnas existentes
            cols_order = [col for col in cols_order if col in comparison_df.columns]
            comparison_df = comparison_df[cols_order]

            results_by_model[model_name] = comparison_df

            print(f"\n{'='*80}")
            print(f"Resultados para modelo: {model_name}")
            print(f"{'='*80}")
            print(comparison_df.to_string(index=False))
            print()

    return results_by_model


def save_comparison_results(results_dict, output_folder):
    """
    Guarda las tablas de comparación en archivos CSV y Excel.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar los resultados
    """

    os.makedirs(output_folder, exist_ok=True)

    # Guardar cada modelo por separado
    for model_name, df in results_dict.items():
        csv_path = os.path.join(output_folder, f'{model_name}_comparison.csv')
        df.to_csv(csv_path, index=False)
        print(f"Guardado: {csv_path}")

    # Guardar todos los modelos en un único Excel con hojas separadas
    excel_path = os.path.join(output_folder, 'all_models_comparison.xlsx')
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for model_name, df in results_dict.items():
            df.to_excel(writer, sheet_name=model_name, index=False)

    print(f"Guardado: {excel_path}")


def create_publication_ready_tables(results_dict, output_folder, dpi=300):
    """
    Crea tablas con formato académico profesional para inclusión en TFM/tesis.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar las tablas
        dpi: Resolución de las imágenes (300 para publicación)
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']
    plt.rcParams['font.size'] = 10

    for model_name, df in results_dict.items():

        # Preparar datos para visualización
        display_df = df.copy()

        # Renombrar columnas para formato académico en español
        column_mapping = {
            'sensor_subset': 'Subconjunto de Sensores',
            'test_acc': 'Exactitud Test (%)',
            'test_bal_acc': 'Exactitud Balanceada Test (%)',
            'test_f1_acc': 'F1 Test (%)',
            'train_acc': 'Exactitud Entrenamiento (%)',
            'train_bal_acc': 'Exactitud Balanceada Entrenamiento (%)',
            'train_f1_acc': 'F1 Entrenamiento (%)',
            'test_loss': 'Pérdida Test',
            'train_loss': 'Pérdida Entrenamiento'
        }

        display_df.rename(columns=column_mapping, inplace=True)

        # Crear figura
        fig, ax = plt.subplots(figsize=(16, len(display_df) * 0.6 + 1.5))
        ax.axis('tight')
        ax.axis('off')

        # Crear tabla
        table = ax.table(
            cellText=display_df.values,
            colLabels=display_df.columns,
            cellLoc='center',
            loc='center',
            bbox=[0, 0, 1, 1]
        )

        # Estilo de celdas
        table.auto_set_font_size(False)
        table.set_fontsize(8.5)
        table.scale(1, 2)

        # Formato del encabezado
        for i in range(len(display_df.columns)):
            cell = table[(0, i)]
            cell.set_facecolor('#2C3E50')
            cell.set_text_props(weight='bold', color='white')
            cell.set_edgecolor('white')
            cell.set_linewidth(1.5)

        # Formato de filas alternadas
        for i in range(1, len(display_df) + 1):
            for j in range(len(display_df.columns)):
                cell = table[(i, j)]

                if i % 2 == 0:
                    cell.set_facecolor('#ECF0F1')
                else:
                    cell.set_facecolor('white')

                cell.set_edgecolor('#BDC3C7')
                cell.set_linewidth(0.5)

                # Primera columna en negrita
                if j == 0:
                    cell.set_text_props(weight='bold')

        # Título en español
        plt.title(f'{model_name} - Comparación por Subconjunto de Sensores',
                 fontsize=14, fontweight='bold', pad=20)

        # Guardar
        output_path = os.path.join(output_folder, f'{model_name}_tabla.png')
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight',
                   facecolor='white', edgecolor='none')
        plt.close()

        print(f"Tabla generada: {output_path}")


def create_comparison_heatmap(results_dict, output_folder, metric='test_bal_acc', dpi=300):
    """
    Crea heatmap comparativo de métricas entre modelos y subsets.

    Args:
        results_dict: Diccionario con DataFrames por modelo
        output_folder: Carpeta donde guardar el heatmap
        metric: Métrica a visualizar
        dpi: Resolución de la imagen
    """

    os.makedirs(output_folder, exist_ok=True)

    # Diccionario de traducción de métricas
    metric_names = {
        'test_acc': 'Exactitud Test',
        'test_bal_acc': 'Exactitud Balanceada Test',
        'test_f1_acc': 'F1 Score Test',
        'train_acc': 'Exactitud Entrenamiento',
        'train_bal_acc': 'Exactitud Balanceada Entrenamiento',
        'train_f1_acc': 'F1 Score Entrenamiento'
    }

    # Extraer valores numéricos de la métrica
    heatmap_data = {}

    for model_name, df in results_dict.items():
        if metric in df.columns:
            # Extraer solo el valor medio (antes del ±)
            values = df[metric].apply(lambda x: float(x.split('±')[0].strip()))
            heatmap_data[model_name] = values.values

    if not heatmap_data:
        print(f"Métrica {metric} no encontrada")
        return

    # Crear DataFrame para heatmap
    sensor_subsets = results_dict[list(results_dict.keys())[0]]['sensor_subset'].values
    heatmap_df = pd.DataFrame(heatmap_data, index=sensor_subsets)

    # Crear figura
    plt.figure(figsize=(12, 6))

    # Configurar estilo
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Configurar colormap
    sns.heatmap(heatmap_df, annot=True, fmt='.2f', cmap='RdYlGn',
               center=heatmap_df.values.mean(),
               cbar_kws={'label': metric_names.get(metric, metric)},
               linewidths=0.5, linecolor='gray')

    metric_title = metric_names.get(metric, metric)
    plt.title(f'Comparación de {metric_title} entre Modelos y Subconjuntos de Sensores',
             fontsize=13, fontweight='bold', pad=15)
    plt.xlabel('Modelo', fontsize=12, fontweight='bold')
    plt.ylabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    plt.tight_layout()

    output_path = os.path.join(output_folder, f'mapa_calor_{metric}.png')
    plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"Mapa de calor generado: {output_path}")


def create_consolidated_latex_table(results_dict, output_folder):
    """
    Crea tabla consolidada en formato LaTeX para inclusión directa en documento.

    Args:
        results_dict: Diccionario con DataFrames por modelo
        output_folder: Carpeta donde guardar el archivo LaTeX
    """

    os.makedirs(output_folder, exist_ok=True)

    # Diccionario de traducción para LaTeX
    translation_dict = {
        'Sensor Subset': 'Subconjunto de Sensores',
        'Test Acc': 'Exactitud Test',
        'Test Bal Acc': 'Exactitud Bal. Test',
        'Test F1 Acc': 'F1 Test',
        'Train Acc': 'Exactitud Entren.',
        'Train Bal Acc': 'Exactitud Bal. Entren.',
        'Train F1 Acc': 'F1 Entren.',
        'Test Loss': 'Pérdida Test',
        'Train Loss': 'Pérdida Entren.'
    }

    for model_name, df in results_dict.items():

        # Preparar datos
        display_df = df.copy()

        # Renombrar columnas
        display_df.columns = [col.replace('_', ' ').title() for col in display_df.columns]
        display_df.columns = [translation_dict.get(col, col) for col in display_df.columns]

        # Convertir a LaTeX
        latex_str = display_df.to_latex(
            index=False,
            column_format='l' + 'c' * (len(display_df.columns) - 1),
            caption=f'Comparación de rendimiento del modelo {model_name} en diferentes subconjuntos de sensores.',
            label=f'tab:{model_name.lower()}_comparacion',
            escape=False,
            position='htbp'
        )

        # Mejorar formato LaTeX
        latex_str = latex_str.replace('\\toprule', '\\hline\\hline')
        latex_str = latex_str.replace('\\midrule', '\\hline')
        latex_str = latex_str.replace('\\bottomrule', '\\hline\\hline')

        # Guardar
        output_path = os.path.join(output_folder, f'{model_name}_tabla.tex')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(latex_str)

        print(f"Tabla LaTeX generada: {output_path}")


def create_bar_comparison(results_dict, output_folder, metric='test_bal_acc', dpi=300):
    """
    Crea gráfico de barras comparativo por métrica.

    Args:
        results_dict: Diccionario con DataFrames por modelo
        output_folder: Carpeta donde guardar el gráfico
        metric: Métrica a visualizar
        dpi: Resolución de la imagen
    """

    os.makedirs(output_folder, exist_ok=True)

    # Diccionario de traducción de métricas
    metric_names = {
        'test_acc': 'Exactitud Test (%)',
        'test_bal_acc': 'Exactitud Balanceada Test (%)',
        'test_f1_acc': 'F1 Score Test (%)',
        'train_acc': 'Exactitud Entrenamiento (%)',
        'train_bal_acc': 'Exactitud Balanceada Entrenamiento (%)',
        'train_f1_acc': 'F1 Score Entrenamiento (%)'
    }

    # Extraer datos
    plot_data = []

    for model_name, df in results_dict.items():
        if metric in df.columns:
            for _, row in df.iterrows():
                mean_val = float(row[metric].split('±')[0].strip())
                std_val = float(row[metric].split('±')[1].strip())

                plot_data.append({
                    'Modelo': model_name,
                    'Subconjunto de Sensores': row['sensor_subset'],
                    'Media': mean_val,
                    'Desviación': std_val
                })

    plot_df = pd.DataFrame(plot_data)

    # Configurar estilo
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Crear gráfico
    fig, ax = plt.subplots(figsize=(14, 6))

    sensor_subsets = plot_df['Subconjunto de Sensores'].unique()
    x = np.arange(len(sensor_subsets))
    width = 0.8 / len(results_dict)

    colors = plt.cm.Set3(np.linspace(0, 1, len(results_dict)))

    for i, (model_name, color) in enumerate(zip(results_dict.keys(), colors)):
        model_data = plot_df[plot_df['Modelo'] == model_name]
        means = model_data['Media'].values
        stds = model_data['Desviación'].values

        bars = ax.bar(x + i * width, means, width, yerr=stds,
                     label=model_name, color=color, capsize=3,
                     edgecolor='black', linewidth=0.5)

        # Añadir valores sobre las barras
        for bar, mean in zip(bars, means):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{mean:.1f}',
                   ha='center', va='bottom', fontsize=8)

    metric_label = metric_names.get(metric, metric)
    ax.set_xlabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
    ax.set_ylabel(metric_label, fontsize=12, fontweight='bold')
    ax.set_title(f'Comparación de {metric_label} entre Modelos',
                fontsize=14, fontweight='bold')
    ax.set_xticks(x + width * (len(results_dict) - 1) / 2)
    ax.set_xticklabels(sensor_subsets, rotation=45, ha='right')
    ax.legend(loc='best', frameon=True, shadow=True, title='Modelo')
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    plt.tight_layout()

    output_path = os.path.join(output_folder, f'comparacion_barras_{metric}.png')
    plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"Gráfico de barras generado: {output_path}")


# ============================================================================
# EJECUCIÓN PRINCIPAL
# ============================================================================

# Definir ruta base de resultados
FINAL_BASE_FOLDER = os.path.join(
    '/content/drive/My Drive/TFM_2/resultados_entrenamiento/',
    '2025-12-08',
    'Fase_1',
    '2025-12-08_00-27-50',
    'resultados_entrenamiento'
)

# Extraer y comparar resultados
results = extract_and_compare_results(
    base_folder=FINAL_BASE_FOLDER,
    model_list=model_list,
    sensor_subset_list=sensor_subset_list,
    ground_truth='stroke_type',
    technique='no-technique'
)

# Guardar comparaciones básicas
comparison_output = os.path.join(FINAL_BASE_FOLDER, 'comparaciones')
save_comparison_results(results, comparison_output)

print(f"\n{'#'*80}")
print(f"# COMPARACIÓN COMPLETADA")
print(f"# Resultados guardados en: {comparison_output}")
print(f"{'#'*80}\n")

# ============================================================================
# GENERAR VISUALIZACIONES ACADÉMICAS
# ============================================================================

viz_output = os.path.join(FINAL_BASE_FOLDER, 'visualizaciones_academicas')

print("\n" + "="*80)
print("GENERANDO VISUALIZACIONES ACADÉMICAS")
print("="*80 + "\n")

# Tablas con formato profesional
create_publication_ready_tables(results, viz_output, dpi=300)

# Mapas de calor de métricas clave
for metric in ['test_bal_acc', 'test_f1_acc', 'test_acc']:
    create_comparison_heatmap(results, viz_output, metric=metric, dpi=300)

# Gráficos de barras
for metric in ['test_bal_acc', 'test_f1_acc','test_acc']:
    create_bar_comparison(results, viz_output, metric=metric, dpi=300)

# Tablas LaTeX
create_consolidated_latex_table(results, viz_output)

print("\n" + "#"*80)
print("# VISUALIZACIONES COMPLETADAS")
print(f"# Archivos guardados en: {viz_output}")
print("#"*80 + "\n")


Resultados para modelo: LSTM
 sensor_subset      test_acc  test_bal_acc   test_f1_acc     train_acc train_bal_acc  train_f1_acc       test_loss      train_loss
    allStreams 90.85 ± 14.48 88.78 ± 16.70 90.40 ± 15.17 100.00 ± 0.02 100.00 ± 0.02 100.00 ± 0.02 0.4645 ± 1.0184 0.0003 ± 0.0005
    onlyGforce 54.13 ± 22.36 49.59 ± 21.05 51.86 ± 23.13  86.80 ± 3.24  84.61 ± 3.94  86.65 ± 3.32 2.4161 ± 2.5970 0.3173 ± 0.0726
onlyCognionics 51.06 ± 12.45 40.77 ± 13.16 43.87 ± 14.07  71.44 ± 4.12  67.35 ± 4.77  70.83 ± 4.35 2.1276 ± 2.1345 0.6054 ± 0.0681
       onlyEye 75.95 ± 20.32 74.04 ± 20.81 75.36 ± 20.68  92.00 ± 2.99  91.09 ± 3.34  91.99 ± 3.00 0.9877 ± 1.4008 0.2072 ± 0.0722
    onlyInsole 81.44 ± 14.10 77.50 ± 16.09 79.70 ± 15.35  97.41 ± 1.37  97.00 ± 1.62  97.41 ± 1.38 0.7572 ± 0.7920 0.0772 ± 0.0382
      onlyBody 89.69 ± 16.23 88.24 ± 18.11 89.16 ± 17.11 100.00 ± 0.00 100.00 ± 0.01 100.00 ± 0.00 0.5974 ± 1.1677 0.0011 ± 0.0003


Resultados para modelo: ConvLSTM
 sensor_subset    

In [ ]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def get_model_used_parameters(model_name):
    """
    Retorna los parámetros que realmente utiliza cada modelo.

    Args:
        model_name: Nombre del modelo

    Returns:
        list: Lista de nombres de parámetros utilizados por el modelo
    """

    # Parámetros comunes a todos los modelos (excepto Baseline)
    common_params = ['batch_size', 'hidden_features', 'learning_rate']

    # Parámetros específicos por modelo
    model_specific_params = {
        'Baseline': [],  # No usa hiperparámetros
        'Conv1D': common_params,
        'LSTM': common_params,
        'ConvLSTM': common_params,
        'Transformer': common_params,
        'ImprovedConvLSTMModel': common_params + ['lstm_hidden', 'reduction_ratio', 'dropout_rate'],
        'ResNetLSTMModel': common_params + ['lstm_hidden', 'num_blocks', 'dropout_rate']
    }

    return model_specific_params.get(model_name, common_params)


def extract_hpo_parameters(optuna_base_folder, model_list, sensor_subset_list,
                          ground_truth='stroke_type', technique='no-technique'):
    """
    Extrae los mejores hiperparámetros encontrados por Optuna para cada configuración.
    Solo incluye los parámetros realmente utilizados por cada modelo.

    Args:
        optuna_base_folder: Carpeta base donde se guardaron los resultados de HPO
        model_list: Lista de nombres de modelos
        sensor_subset_list: Lista de subconjuntos de sensores
        ground_truth: Variable objetivo utilizada
        technique: Técnica utilizada

    Returns:
        dict: Diccionario con DataFrames de hiperparámetros por modelo
    """

    params_by_model = {}

    for model_name in model_list:
        param_data = []

        # Obtener parámetros utilizados por este modelo
        used_params = get_model_used_parameters(model_name)

        for subset in sensor_subset_list:
            # Construir ruta al archivo de configuración óptima
            config_folder = f'optuna_{model_name}_{technique}_{ground_truth}_{subset}'
            best_config_path = os.path.join(
                optuna_base_folder,
                config_folder,
                'best_config.json'
            )

            # Verificar si existe el archivo
            if not os.path.exists(best_config_path):
                print(f"Advertencia: No se encontró {best_config_path}")
                continue

            # Leer configuración óptima
            with open(best_config_path, 'r') as f:
                best_config = json.load(f)

            # Extraer parámetros
            params_dict = {'sensor_subset': subset}

            # Añadir valor de la métrica optimizada
            params_dict['best_value'] = f"{best_config['best_value']:.4f}"

            # Añadir solo los hiperparámetros utilizados por este modelo
            for param_name in used_params:
                if param_name in best_config['best_params']:
                    param_value = best_config['best_params'][param_name]
                    if isinstance(param_value, float):
                        params_dict[param_name] = f"{param_value:.4f}"
                    else:
                        params_dict[param_name] = str(param_value)

            param_data.append(params_dict)

        # Crear DataFrame de parámetros
        if param_data:
            params_df = pd.DataFrame(param_data)

            # Reordenar columnas: sensor_subset primero, luego best_value, luego el resto
            cols = ['sensor_subset', 'best_value']
            other_cols = [col for col in params_df.columns if col not in cols]
            params_df = params_df[cols + sorted(other_cols)]

            params_by_model[model_name] = params_df

            print(f"\n{'='*80}")
            print(f"Hiperparámetros óptimos para modelo: {model_name}")
            print(f"{'='*80}")
            print(params_df.to_string(index=False))
            print()

    return params_by_model


def create_hpo_parameters_table(params_dict, output_folder, dpi=300):
    """
    Crea tablas visuales con los hiperparámetros óptimos para cada modelo.
    Cada tabla muestra solo los parámetros relevantes para ese modelo.

    Args:
        params_dict: Diccionario con DataFrames de parámetros por modelo
        output_folder: Carpeta donde guardar las tablas
        dpi: Resolución de las imágenes
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']
    plt.rcParams['font.size'] = 9

    # Diccionario de traducción de nombres de parámetros
    param_translation = {
        'sensor_subset': 'Subconjunto de Sensores',
        'best_value': 'Métrica Óptima',
        'batch_size': 'Tamaño de Lote',
        'hidden_features': 'Características Ocultas',
        'learning_rate': 'Tasa de Aprendizaje',
        'lstm_hidden': 'Unidades LSTM',
        'dropout_rate': 'Tasa de Dropout',
        'reduction_ratio': 'Ratio de Reducción',
        'use_attention': 'Usar Atención',
        'num_blocks': 'Número de Bloques'
    }

    for model_name, df in params_dict.items():

        # Preparar datos para visualización
        display_df = df.copy()

        # Traducir nombres de columnas
        display_df.columns = [param_translation.get(col, col) for col in display_df.columns]

        # Ajustar tamaño de figura según número de columnas y filas
        n_cols = len(display_df.columns)
        n_rows = len(display_df)
        fig_width = max(10, n_cols * 2.0)
        fig_height = max(4, n_rows * 0.6 + 1.5)

        # Crear figura
        fig, ax = plt.subplots(figsize=(fig_width, fig_height))
        ax.axis('tight')
        ax.axis('off')

        # Crear tabla
        table = ax.table(
            cellText=display_df.values,
            colLabels=display_df.columns,
            cellLoc='center',
            loc='center',
            bbox=[0, 0, 1, 1]
        )

        # Estilo de celdas
        table.auto_set_font_size(False)
        table.set_fontsize(8.5)
        table.scale(1, 2.2)

        # Formato del encabezado
        for i in range(len(display_df.columns)):
            cell = table[(0, i)]
            cell.set_facecolor('#34495E')
            cell.set_text_props(weight='bold', color='white')
            cell.set_edgecolor('white')
            cell.set_linewidth(1.5)

        # Formato de filas alternadas
        for i in range(1, len(display_df) + 1):
            for j in range(len(display_df.columns)):
                cell = table[(i, j)]

                if i % 2 == 0:
                    cell.set_facecolor('#ECF0F1')
                else:
                    cell.set_facecolor('white')

                cell.set_edgecolor('#BDC3C7')
                cell.set_linewidth(0.5)

                # Primera columna en negrita
                if j == 0:
                    cell.set_text_props(weight='bold')

                # Segunda columna (métrica óptima) en negrita y verde
                if j == 1:
                    cell.set_text_props(weight='bold', color='#27AE60')

        # Título
        plt.title(f'{model_name} - Hiperparámetros Óptimos (HPO)',
                 fontsize=14, fontweight='bold', pad=20)

        # Guardar
        output_path = os.path.join(output_folder, f'{model_name}_hiperparametros.png')
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight',
                   facecolor='white', edgecolor='none')
        plt.close()

        print(f"Tabla de hiperparámetros generada: {output_path}")


def create_hpo_latex_table(params_dict, output_folder):
    """
    Crea tablas LaTeX con los hiperparámetros óptimos.
    Cada tabla muestra solo los parámetros relevantes para ese modelo.

    Args:
        params_dict: Diccionario con DataFrames de parámetros por modelo
        output_folder: Carpeta donde guardar las tablas LaTeX
    """

    os.makedirs(output_folder, exist_ok=True)

    # Diccionario de traducción
    param_translation = {
        'Sensor Subset': 'Subconjunto de Sensores',
        'Best Value': 'Métrica Óptima',
        'Batch Size': 'Tamaño de Lote',
        'Dropout Rate': 'Tasa de Dropout',
        'Hidden Features': 'Caract. Ocultas',
        'Learning Rate': 'Tasa de Aprend.',
        'Lstm Hidden': 'Unidades LSTM',
        'Num Blocks': 'Núm. Bloques',
        'Reduction Ratio': 'Ratio de Red.',
        'Use Attention': 'Usar Atención'
    }

    for model_name, df in params_dict.items():

        # Preparar datos
        display_df = df.copy()

        # Renombrar columnas para LaTeX
        display_df.columns = [col.replace('_', ' ').title() for col in display_df.columns]
        display_df.columns = [param_translation.get(col, col) for col in display_df.columns]

        # Convertir a LaTeX
        latex_str = display_df.to_latex(
            index=False,
            column_format='l' + 'c' * (len(display_df.columns) - 1),
            caption=f'Hiperparámetros óptimos encontrados por HPO para el modelo {model_name}.',
            label=f'tab:{model_name.lower()}_hiperparametros',
            escape=False,
            position='htbp'
        )

        # Mejorar formato LaTeX
        latex_str = latex_str.replace('\\toprule', '\\hline\\hline')
        latex_str = latex_str.replace('\\midrule', '\\hline')
        latex_str = latex_str.replace('\\bottomrule', '\\hline\\hline')

        # Guardar
        output_path = os.path.join(output_folder, f'{model_name}_hiperparametros.tex')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(latex_str)

        print(f"Tabla LaTeX de hiperparámetros generada: {output_path}")


def create_hpo_comparison_heatmap(params_dict, output_folder, param_name='learning_rate', dpi=300):
    """
    Crea heatmap comparativo de un hiperparámetro específico entre modelos y subsets.
    Solo incluye modelos que realmente usan ese parámetro.

    Args:
        params_dict: Diccionario con DataFrames de parámetros por modelo
        output_folder: Carpeta donde guardar el heatmap
        param_name: Nombre del hiperparámetro a visualizar
        dpi: Resolución de la imagen
    """

    os.makedirs(output_folder, exist_ok=True)

    # Diccionario de traducción de parámetros
    param_translation = {
        'learning_rate': 'Tasa de Aprendizaje',
        'batch_size': 'Tamaño de Lote',
        'hidden_features': 'Características Ocultas',
        'lstm_hidden': 'Unidades LSTM',
        'dropout_rate': 'Tasa de Dropout',
        'num_blocks': 'Número de Bloques',
        'reduction_ratio': 'Ratio de Reducción'
    }

    # Extraer valores del parámetro (solo de modelos que lo usan)
    heatmap_data = {}

    for model_name, df in params_dict.items():
        if param_name in df.columns:
            # Convertir valores a numéricos
            try:
                values = df[param_name].apply(lambda x: float(x) if isinstance(x, str) else x)
                heatmap_data[model_name] = values.values
            except:
                print(f"No se pudo convertir {param_name} a numérico para {model_name}")
                continue

    if not heatmap_data:
        print(f"Parámetro {param_name} no encontrado en ningún modelo")
        return

    if len(heatmap_data) < 2:
        print(f"Parámetro {param_name} solo usado por un modelo, omitiendo heatmap")
        return

    # Crear DataFrame para heatmap
    sensor_subsets = params_dict[list(heatmap_data.keys())[0]]['sensor_subset'].values
    heatmap_df = pd.DataFrame(heatmap_data, index=sensor_subsets)

    # Configurar estilo
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Crear figura
    fig_width = max(10, len(heatmap_data) * 2)
    plt.figure(figsize=(fig_width, 6))

    # Configurar colormap
    sns.heatmap(heatmap_df, annot=True, fmt='.4f', cmap='viridis',
               cbar_kws={'label': param_translation.get(param_name, param_name)},
               linewidths=0.5, linecolor='gray')

    param_label = param_translation.get(param_name, param_name)
    plt.title(f'Comparación de {param_label} Seleccionado por HPO',
             fontsize=13, fontweight='bold', pad=15)
    plt.xlabel('Modelo', fontsize=12, fontweight='bold')
    plt.ylabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    plt.tight_layout()

    output_path = os.path.join(output_folder, f'mapa_calor_hpo_{param_name}.png')
    plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"Mapa de calor de hiperparámetro generado: {output_path}")


def save_hpo_parameters_excel(params_dict, output_folder):
    """
    Guarda los hiperparámetros en un archivo Excel con hojas por modelo.

    Args:
        params_dict: Diccionario con DataFrames de parámetros por modelo
        output_folder: Carpeta donde guardar el archivo
    """

    os.makedirs(output_folder, exist_ok=True)

    excel_path = os.path.join(output_folder, 'hiperparametros_optimos_hpo.xlsx')

    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for model_name, df in params_dict.items():
            df.to_excel(writer, sheet_name=model_name, index=False)

    print(f"Archivo Excel de hiperparámetros guardado: {excel_path}")


def create_parameter_usage_summary(params_dict, output_folder):
    """
    Crea una tabla resumen mostrando qué parámetros usa cada modelo.

    Args:
        params_dict: Diccionario con DataFrames de parámetros por modelo
        output_folder: Carpeta donde guardar la tabla
    """

    os.makedirs(output_folder, exist_ok=True)

    # Obtener todos los parámetros únicos
    all_params = set()
    for df in params_dict.values():
        all_params.update([col for col in df.columns if col not in ['sensor_subset', 'best_value']])

    all_params = sorted(list(all_params))

    # Crear matriz de uso
    usage_data = []
    for model_name in params_dict.keys():
        model_params = params_dict[model_name].columns.tolist()
        row = {'Modelo': model_name}
        for param in all_params:
            row[param] = '✓' if param in model_params else '-'
        usage_data.append(row)

    usage_df = pd.DataFrame(usage_data)

    # Traducción de parámetros
    param_translation = {
        'batch_size': 'Tamaño de Lote',
        'hidden_features': 'Caract. Ocultas',
        'learning_rate': 'Tasa de Aprend.',
        'lstm_hidden': 'Unidades LSTM',
        'dropout_rate': 'Tasa de Dropout',
        'reduction_ratio': 'Ratio de Red.',
        'num_blocks': 'Núm. Bloques'
    }

    usage_df.columns = ['Modelo'] + [param_translation.get(col, col) for col in usage_df.columns[1:]]

    # Crear visualización
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    fig, ax = plt.subplots(figsize=(12, len(usage_df) * 0.8 + 1))
    ax.axis('tight')
    ax.axis('off')

    table = ax.table(
        cellText=usage_df.values,
        colLabels=usage_df.columns,
        cellLoc='center',
        loc='center',
        bbox=[0, 0, 1, 1]
    )

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)

    # Formato del encabezado
    for i in range(len(usage_df.columns)):
        cell = table[(0, i)]
        cell.set_facecolor('#34495E')
        cell.set_text_props(weight='bold', color='white')
        cell.set_edgecolor('white')
        cell.set_linewidth(1.5)

    # Formato de celdas
    for i in range(1, len(usage_df) + 1):
        for j in range(len(usage_df.columns)):
            cell = table[(i, j)]

            if i % 2 == 0:
                cell.set_facecolor('#ECF0F1')
            else:
                cell.set_facecolor('white')

            cell.set_edgecolor('#BDC3C7')
            cell.set_linewidth(0.5)

            # Primera columna en negrita
            if j == 0:
                cell.set_text_props(weight='bold')

            # Colorear checkmarks
            if cell.get_text().get_text() == '✓':
                cell.set_text_props(color='#27AE60', weight='bold')

    plt.title('Uso de Hiperparámetros por Modelo',
             fontsize=14, fontweight='bold', pad=20)

    output_path = os.path.join(output_folder, 'resumen_uso_parametros.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight',
               facecolor='white', edgecolor='none')
    plt.close()

    print(f"Tabla resumen de uso de parámetros generada: {output_path}")


# ============================================================================
# EJECUCIÓN PRINCIPAL - HIPERPARÁMETROS HPO
# ============================================================================

# Definir ruta base de resultados de Optuna
OPTUNA_BASE_FOLDER = os.path.join(
    '/content/drive/My Drive/TFM_2/resultados_entrenamiento/',
    '2025-12-08',
    'Fase_1',
    '2025-12-08_00-27-50',
    'HPO'
)

# Extraer hiperparámetros óptimos (solo los usados por cada modelo)
hpo_params = extract_hpo_parameters(
    optuna_base_folder=OPTUNA_BASE_FOLDER,
    model_list=model_list,
    sensor_subset_list=sensor_subset_list,
    ground_truth='stroke_type',
    technique='no-technique'
)

# Crear carpeta de visualizaciones de HPO
hpo_viz_output = os.path.join(FINAL_BASE_FOLDER, 'visualizaciones_hpo')

print("\n" + "="*80)
print("GENERANDO VISUALIZACIONES DE HIPERPARÁMETROS HPO")
print("="*80 + "\n")

# Tabla resumen de uso de parámetros
create_parameter_usage_summary(hpo_params, hpo_viz_output)

# Tablas visuales de hiperparámetros (solo parámetros usados)
create_hpo_parameters_table(hpo_params, hpo_viz_output, dpi=300)

# Tablas LaTeX
create_hpo_latex_table(hpo_params, hpo_viz_output)

# Mapas de calor para hiperparámetros (solo para modelos que los usan)
for param in ['learning_rate', 'dropout_rate', 'hidden_features', 'lstm_hidden', 'num_blocks']:
    create_hpo_comparison_heatmap(hpo_params, hpo_viz_output, param_name=param, dpi=300)

# Guardar en Excel
save_hpo_parameters_excel(hpo_params, hpo_viz_output)

print("\n" + "#"*80)
print("# VISUALIZACIONES DE HIPERPARÁMETROS COMPLETADAS")
print(f"# Archivos guardados en: {hpo_viz_output}")
print("#"*80 + "\n")


Hiperparámetros óptimos para modelo: LSTM
 sensor_subset best_value batch_size hidden_features learning_rate
    allStreams     0.9890         64              64        0.0018
    onlyGforce     0.6282         64              64        0.0018
onlyCognionics     0.7064        128             128        0.0153
       onlyEye     0.8863         64              64        0.0036
    onlyInsole     0.8277        128             128        0.0153
      onlyBody     0.9845        128             128        0.0153


Hiperparámetros óptimos para modelo: ConvLSTM
 sensor_subset best_value batch_size hidden_features learning_rate
    allStreams     0.9481        128             128        0.0153
    onlyGforce     0.5309         64             128        0.0012
onlyCognionics     0.6289        128             128        0.0153
       onlyEye     0.8449         64             128        0.0041
    onlyInsole     0.7709        128             128        0.0153
      onlyBody     0.9796         64  

In [ ]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def create_metrics_heatmap_per_model(results_dict, output_folder, dpi=300):
    """
    Crea heatmaps separados para cada modelo: uno de métricas de precisión y otro de pérdida.
    Ahora incluye la desviación estándar en las anotaciones.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar los heatmaps
        dpi: Resolución de las imágenes
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Diccionario de traducción de métricas
    metric_translation = {
        'test_acc': 'Exactitud Test',
        'test_bal_acc': 'Exactitud Bal. Test',
        'test_f1_acc': 'F1 Test',
        'train_acc': 'Exactitud Entren.',
        'train_bal_acc': 'Exactitud Bal. Entren.',
        'train_f1_acc': 'F1 Entren.',
        'test_loss': 'Pérdida Test',
        'train_loss': 'Pérdida Entren.'
    }

    for model_name, df in results_dict.items():

        # Preparar datos para el heatmap
        heatmap_data = []
        heatmap_annotations = []
        metrics_columns = []

        # Extraer valores numéricos y desviaciones estándar de cada métrica
        for col in df.columns:
            if col != 'sensor_subset':
                # Extraer valor medio y desviación estándar
                means = []
                stds = []
                annotations = []

                for val in df[col]:
                    if '±' in str(val):
                        parts = str(val).split('±')
                        mean = float(parts[0].strip())
                        std = float(parts[1].strip())
                    else:
                        mean = float(str(val).strip())
                        std = 0.0

                    means.append(mean)
                    stds.append(std)
                    annotations.append(f"{mean:.2f}\n±{std:.2f}")

                heatmap_data.append(means)
                heatmap_annotations.append(annotations)
                metrics_columns.append(metric_translation.get(col, col))

        # Crear DataFrame para heatmap (transpuesto)
        sensor_subsets = df['sensor_subset'].values
        heatmap_df = pd.DataFrame(
            np.array(heatmap_data).T,
            index=sensor_subsets,
            columns=metrics_columns
        )

        # Crear matriz de anotaciones (transpuesta)
        annotation_matrix = np.array(heatmap_annotations).T

        # Separar métricas de accuracy/F1 de métricas de loss
        accuracy_metrics = [col for col in heatmap_df.columns if 'Pérdida' not in col]
        loss_metrics = [col for col in heatmap_df.columns if 'Pérdida' in col]

        # ========== HEATMAP 1: MÉTRICAS DE PRECISIÓN ==========
        if accuracy_metrics:
            accuracy_indices = [i for i, col in enumerate(metrics_columns) if col in accuracy_metrics]
            accuracy_annotations = annotation_matrix[:, accuracy_indices]

            fig, ax = plt.subplots(figsize=(14, max(7, len(sensor_subsets) * 0.6)))

            sns.heatmap(heatmap_df[accuracy_metrics],
                       annot=accuracy_annotations,
                       fmt='',
                       cmap='RdYlGn',
                       center=None,
                       linewidths=0.8,
                       linecolor='white',
                       cbar_kws={'label': 'Valor Medio (%)'},
                       ax=ax,
                       vmin=heatmap_df[accuracy_metrics].values.min() - 5,
                       vmax=100,
                       annot_kws={'fontsize': 8, 'weight': 'bold'})

            ax.set_xlabel('Métrica', fontsize=12, fontweight='bold')
            ax.set_ylabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontweight='bold')
            ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontweight='bold')

            plt.tight_layout()

            # Guardar heatmap de precisión
            output_path = os.path.join(output_folder, f'{model_name}_heatmap_precision.png')
            plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
            plt.close()

            print(f"Heatmap de precisión generado: {output_path}")

        # ========== HEATMAP 2: MÉTRICAS DE PÉRDIDA ==========
        if loss_metrics:
            loss_indices = [i for i, col in enumerate(metrics_columns) if col in loss_metrics]
            loss_annotations = annotation_matrix[:, loss_indices]

            # Para pérdidas, usar formato con 4 decimales
            loss_annotations_formatted = []
            for row in range(loss_annotations.shape[0]):
                row_annotations = []
                for col_idx, col in enumerate(loss_metrics):
                    original_col_idx = metrics_columns.index(col)
                    val = df[list(df.columns)[original_col_idx + 1]].iloc[row]

                    if '±' in str(val):
                        parts = str(val).split('±')
                        mean = float(parts[0].strip())
                        std = float(parts[1].strip())
                        row_annotations.append(f"{mean:.4f}\n±{std:.4f}")
                    else:
                        mean = float(str(val).strip())
                        row_annotations.append(f"{mean:.4f}\n±0.0000")

                loss_annotations_formatted.append(row_annotations)

            loss_annotations_formatted = np.array(loss_annotations_formatted)

            fig, ax = plt.subplots(figsize=(10, max(7, len(sensor_subsets) * 0.6)))

            sns.heatmap(heatmap_df[loss_metrics],
                       annot=loss_annotations_formatted,
                       fmt='',
                       cmap='RdYlGn_r',
                       center=None,
                       linewidths=0.8,
                       linecolor='white',
                       cbar_kws={'label': 'Valor Medio'},
                       ax=ax,
                       annot_kws={'fontsize': 8, 'weight': 'bold'})

            ax.set_xlabel('Métrica', fontsize=12, fontweight='bold')
            ax.set_ylabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontweight='bold')
            ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontweight='bold')

            plt.tight_layout()

            # Guardar heatmap de pérdida
            output_path = os.path.join(output_folder, f'{model_name}_heatmap_perdida.png')
            plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
            plt.close()

            print(f"Heatmap de pérdida generado: {output_path}")


def create_single_combined_heatmap(results_dict, output_folder, dpi=300):
    """
    Crea un único heatmap combinado mostrando todas las métricas de test
    para todos los modelos y subconjuntos de sensores.
    Ahora incluye la desviación estándar en las anotaciones.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar el heatmap
        dpi: Resolución de la imagen
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Métricas de test a incluir
    test_metrics = ['test_acc', 'test_bal_acc', 'test_f1_acc']

    # Diccionario de traducción
    metric_translation = {
        'test_acc': 'Exactitud',
        'test_bal_acc': 'Exactitud Bal.',
        'test_f1_acc': 'F1 Score'
    }

    # Preparar datos combinados
    combined_data = []
    combined_annotations = []
    index_labels = []

    for model_name, df in results_dict.items():
        for _, row in df.iterrows():
            subset = row['sensor_subset']
            label = f"{model_name} - {subset}"
            index_labels.append(label)

            row_data = []
            row_annotations = []

            for metric in test_metrics:
                if metric in df.columns:
                    val = str(row[metric])
                    if '±' in val:
                        parts = val.split('±')
                        mean = float(parts[0].strip())
                        std = float(parts[1].strip())
                        row_data.append(mean)
                        row_annotations.append(f"{mean:.2f}\n±{std:.2f}")
                    else:
                        mean = float(val.strip())
                        row_data.append(mean)
                        row_annotations.append(f"{mean:.2f}\n±0.00")
                else:
                    row_data.append(np.nan)
                    row_annotations.append("")

            combined_data.append(row_data)
            combined_annotations.append(row_annotations)

    # Crear DataFrames
    combined_df = pd.DataFrame(
        combined_data,
        index=index_labels,
        columns=[metric_translation.get(m, m) for m in test_metrics]
    )

    annotation_matrix = np.array(combined_annotations)

    # Crear figura
    fig_height = max(14, len(index_labels) * 0.5)
    fig, ax = plt.subplots(figsize=(12, fig_height))

    # Crear heatmap con anotaciones que incluyen desviación estándar
    sns.heatmap(combined_df,
               annot=annotation_matrix,
               fmt='',
               cmap='RdYlGn',
               center=None,
               linewidths=0.8,
               linecolor='white',
               cbar_kws={'label': 'Valor Medio (%)'},
               ax=ax,
               vmin=combined_df.values.min() - 5,
               vmax=100,
               annot_kws={'fontsize': 7, 'weight': 'bold'})

    ax.set_xlabel('Métrica', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo - Subconjunto de Sensores', fontsize=12, fontweight='bold')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontweight='bold')
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=8, fontweight='bold')

    plt.tight_layout()

    # Guardar
    output_path = os.path.join(output_folder, 'heatmap_general_todos_modelos.png')
    plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"Heatmap general combinado generado: {output_path}")


def create_metrics_comparison_per_subset(results_dict, output_folder, dpi=300):
    """
    Crea heatmaps mostrando cómo cada subconjunto de sensores se comporta
    en diferentes modelos para cada métrica.
    Ahora incluye la desviación estándar en las anotaciones.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar los heatmaps
        dpi: Resolución de la imagen
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Obtener todos los subconjuntos de sensores
    all_subsets = results_dict[list(results_dict.keys())[0]]['sensor_subset'].values

    # Métricas a visualizar
    metrics_to_plot = {
        'test_acc': 'Exactitud Test (%)',
        'test_bal_acc': 'Exactitud Balanceada Test (%)',
        'test_f1_acc': 'F1 Score Test (%)'
    }

    for metric, metric_label in metrics_to_plot.items():

        # Preparar datos para esta métrica
        heatmap_data = []
        heatmap_annotations = []
        model_names = []

        for model_name, df in results_dict.items():
            if metric in df.columns:
                means = []
                annotations = []

                for val in df[metric]:
                    if '±' in str(val):
                        parts = str(val).split('±')
                        mean = float(parts[0].strip())
                        std = float(parts[1].strip())
                        means.append(mean)
                        annotations.append(f"{mean:.2f}\n±{std:.2f}")
                    else:
                        mean = float(str(val).strip())
                        means.append(mean)
                        annotations.append(f"{mean:.2f}\n±0.00")

                heatmap_data.append(means)
                heatmap_annotations.append(annotations)
                model_names.append(model_name)

        if not heatmap_data:
            continue

        # Crear DataFrames
        heatmap_df = pd.DataFrame(
            np.array(heatmap_data),
            index=model_names,
            columns=all_subsets
        )

        annotation_matrix = np.array(heatmap_annotations)

        # Crear figura
        fig, ax = plt.subplots(figsize=(16, max(7, len(model_names) * 0.9)))

        # Crear heatmap con anotaciones que incluyen desviación estándar
        sns.heatmap(heatmap_df,
                   annot=annotation_matrix,
                   fmt='',
                   cmap='RdYlGn',
                   center=None,
                   linewidths=0.8,
                   linecolor='white',
                   cbar_kws={'label': metric_label},
                   ax=ax,
                   vmin=heatmap_df.values.min() - 5,
                   vmax=100,
                   annot_kws={'fontsize': 8, 'weight': 'bold'})

        ax.set_xlabel('Subconjunto de Sensores', fontsize=12, fontweight='bold')
        ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontweight='bold')
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontweight='bold')

        plt.tight_layout()

        # Guardar
        output_path = os.path.join(output_folder, f'heatmap_por_subset_{metric}.png')
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
        plt.close()

        print(f"Heatmap por subset generado: {output_path}")


# ============================================================================
# EJECUCIÓN - HEATMAPS DE MÉTRICAS
# ============================================================================

timestamp = '2025-12-08_00-27-50'
date_str = '2025-12-08'

FINAL_BASE_FOLDER = os.path.join(
    '/content/drive/My Drive/TFM_2/resultados_entrenamiento/',
    date_str,
    'Fase_1',
    timestamp,
    'resultados_entrenamiento'
)

results = extract_and_compare_results(
    base_folder=FINAL_BASE_FOLDER,
    model_list=model_list,
    sensor_subset_list=sensor_subset_list,
    ground_truth='stroke_type',
    technique='no-technique'
)

# Crear carpeta específica para heatmaps
heatmap_output = os.path.join(FINAL_BASE_FOLDER, 'heatmaps_metricas')

print("\n" + "="*80)
print("GENERANDO HEATMAPS DE MÉTRICAS")
print("="*80 + "\n")

# Heatmaps separados por cada modelo (precisión y pérdida en archivos distintos)
create_metrics_heatmap_per_model(results, heatmap_output, dpi=300)

# Heatmap general combinado (todos los modelos y subsets)
create_single_combined_heatmap(results, heatmap_output, dpi=300)

# Heatmaps por métrica específica (modelos vs subsets)
create_metrics_comparison_per_subset(results, heatmap_output, dpi=300)

print("\n" + "#"*80)
print("# HEATMAPS DE MÉTRICAS COMPLETADOS")
print(f"# Archivos guardados en: {heatmap_output}")
print("#"*80 + "\n")


Resultados para modelo: LSTM
 sensor_subset      test_acc  test_bal_acc   test_f1_acc     train_acc train_bal_acc  train_f1_acc       test_loss      train_loss
    allStreams 90.85 ± 14.48 88.78 ± 16.70 90.40 ± 15.17 100.00 ± 0.02 100.00 ± 0.02 100.00 ± 0.02 0.4645 ± 1.0184 0.0003 ± 0.0005
    onlyGforce 54.13 ± 22.36 49.59 ± 21.05 51.86 ± 23.13  86.80 ± 3.24  84.61 ± 3.94  86.65 ± 3.32 2.4161 ± 2.5970 0.3173 ± 0.0726
onlyCognionics 51.06 ± 12.45 40.77 ± 13.16 43.87 ± 14.07  71.44 ± 4.12  67.35 ± 4.77  70.83 ± 4.35 2.1276 ± 2.1345 0.6054 ± 0.0681
       onlyEye 75.95 ± 20.32 74.04 ± 20.81 75.36 ± 20.68  92.00 ± 2.99  91.09 ± 3.34  91.99 ± 3.00 0.9877 ± 1.4008 0.2072 ± 0.0722
    onlyInsole 81.44 ± 14.10 77.50 ± 16.09 79.70 ± 15.35  97.41 ± 1.37  97.00 ± 1.62  97.41 ± 1.38 0.7572 ± 0.7920 0.0772 ± 0.0382
      onlyBody 89.69 ± 16.23 88.24 ± 18.11 89.16 ± 17.11 100.00 ± 0.00 100.00 ± 0.01 100.00 ± 0.00 0.5974 ± 1.1677 0.0011 ± 0.0003


Resultados para modelo: ConvLSTM
 sensor_subset    

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def create_top_bottom_rankings(results_dict, output_folder, dpi=300):
    """
    Crea gráficas de ranking mostrando los 5 mejores y 5 peores pares modelo-subset
    para cada métrica de test.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar las gráficas
        dpi: Resolución de las imágenes
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Métricas de test a analizar
    test_metrics = {
        'test_acc': 'Exactitud Test (%)',
        'test_bal_acc': 'Exactitud Balanceada Test (%)',
        'test_f1_acc': 'F1 Score Test (%)'
    }

    for metric, metric_label in test_metrics.items():

        # Recopilar todos los pares modelo-subset con sus valores
        all_pairs = []

        for model_name, df in results_dict.items():
            if metric in df.columns:
                for _, row in df.iterrows():
                    subset = row['sensor_subset']
                    value_str = str(row[metric])
                    mean_value = float(value_str.split('±')[0].strip())
                    std_value = float(value_str.split('±')[1].strip()) if '±' in value_str else 0

                    all_pairs.append({
                        'modelo': model_name,
                        'subset': subset,
                        'par': f"{model_name}\n{subset}",
                        'valor': mean_value,
                        'std': std_value
                    })

        # Crear DataFrame y ordenar
        pairs_df = pd.DataFrame(all_pairs)
        pairs_df_sorted = pairs_df.sort_values('valor', ascending=False)

        # Top 5 y Bottom 5
        top_5 = pairs_df_sorted.head(5)
        bottom_5 = pairs_df_sorted.tail(5).sort_values('valor', ascending=False)

        # ========== GRÁFICA TOP 5 ==========
        fig, ax = plt.subplots(figsize=(11, 6))

        # Crear barras horizontales con colores invertidos (mejor = más intenso)
        y_pos = np.arange(len(top_5))
        colors = plt.cm.Greens(np.linspace(0.6, 0.3, len(top_5)))

        bars = ax.barh(y_pos, top_5['valor'].values,
                       color=colors,
                       edgecolor='darkgreen',
                       linewidth=1.2,
                       alpha=0.8)

        # Calcular el máximo valor para ajustar el límite del eje
        max_val = top_5['valor'].max()
        max_std = top_5['std'].max()

        # Añadir valores en las barras (solo números con ±)
        for i, (val, std) in enumerate(zip(top_5['valor'].values, top_5['std'].values)):
            ax.text(val + 0.5, i, f'{val:.2f} ± {std:.2f}',
                   va='center', fontsize=10, fontweight='bold')

        # Configurar ejes con margen adicional
        ax.set_yticks(y_pos)
        ax.set_yticklabels(top_5['par'].values, fontsize=10, fontweight='bold')
        ax.invert_yaxis()
        ax.set_xlabel(metric_label, fontsize=12, fontweight='bold')
        ax.grid(axis='x', alpha=0.3, linestyle='--')

        # Ajustar límite del eje x para que los números no se corten
        ax.set_xlim(0, max_val + max_std + 8)

        plt.tight_layout()

        # Guardar
        output_path = os.path.join(output_folder, f'ranking_top5_{metric}.png')
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
        plt.close()

        print(f"Ranking Top 5 generado: {output_path}")

        # ========== GRÁFICA BOTTOM 5 ==========
        fig, ax = plt.subplots(figsize=(11, 6))

        # Crear barras horizontales con colores invertidos (peor = más intenso)
        y_pos = np.arange(len(bottom_5))
        colors = plt.cm.Reds(np.linspace(0.3, 0.6, len(bottom_5)))

        bars = ax.barh(y_pos, bottom_5['valor'].values,
                       color=colors,
                       edgecolor='darkred',
                       linewidth=1.2,
                       alpha=0.8)

        # Calcular el máximo valor para ajustar el límite del eje
        max_val = bottom_5['valor'].max()
        max_std = bottom_5['std'].max()

        # Añadir valores en las barras (solo números con ±)
        for i, (val, std) in enumerate(zip(bottom_5['valor'].values, bottom_5['std'].values)):
            ax.text(val + 0.5, i, f'{val:.2f} ± {std:.2f}',
                   va='center', fontsize=10, fontweight='bold')

        # Configurar ejes con margen adicional
        ax.set_yticks(y_pos)
        ax.set_yticklabels(bottom_5['par'].values, fontsize=10, fontweight='bold')
        ax.invert_yaxis()
        ax.set_xlabel(metric_label, fontsize=12, fontweight='bold')
        ax.grid(axis='x', alpha=0.3, linestyle='--')

        # Ajustar límite del eje x para que los números no se corten
        ax.set_xlim(0, max_val + max_std + 8)

        plt.tight_layout()

        # Guardar
        output_path = os.path.join(output_folder, f'ranking_bottom5_{metric}.png')
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
        plt.close()

        print(f"Ranking Bottom 5 generado: {output_path}")


def create_combined_top_bottom_ranking(results_dict, output_folder, dpi=300):
    """
    Crea gráficas combinadas mostrando top 5 y bottom 5 en una sola visualización
    para cada métrica.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar las gráficas
        dpi: Resolución de las imágenes
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Métricas de test a analizar
    test_metrics = {
        'test_acc': 'Exactitud Test (%)',
        'test_bal_acc': 'Exactitud Balanceada Test (%)',
        'test_f1_acc': 'F1 Score Test (%)'
    }

    for metric, metric_label in test_metrics.items():

        # Recopilar todos los pares modelo-subset con sus valores
        all_pairs = []

        for model_name, df in results_dict.items():
            if metric in df.columns:
                for _, row in df.iterrows():
                    subset = row['sensor_subset']
                    value_str = str(row[metric])
                    mean_value = float(value_str.split('±')[0].strip())
                    std_value = float(value_str.split('±')[1].strip()) if '±' in value_str else 0

                    all_pairs.append({
                        'modelo': model_name,
                        'subset': subset,
                        'par': f"{model_name}\n{subset}",
                        'valor': mean_value,
                        'std': std_value
                    })

        # Crear DataFrame y ordenar
        pairs_df = pd.DataFrame(all_pairs)
        pairs_df_sorted = pairs_df.sort_values('valor', ascending=False)

        # Top 5 y Bottom 5
        top_5 = pairs_df_sorted.head(5).reset_index(drop=True)
        bottom_5 = pairs_df_sorted.tail(5).sort_values('valor', ascending=False).reset_index(drop=True)

        # ========== GRÁFICA COMBINADA ==========
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

        # --- Subplot 1: TOP 5 ---
        y_pos = np.arange(len(top_5))
        colors_top = plt.cm.Greens(np.linspace(0.6, 0.3, len(top_5)))

        ax1.barh(y_pos, top_5['valor'].values,
                color=colors_top,
                edgecolor='darkgreen',
                linewidth=1.2,
                alpha=0.8)

        # Calcular máximo para ajustar límites
        max_val_top = top_5['valor'].max()
        max_std_top = top_5['std'].max()

        # Valores en las barras
        for i, (val, std) in enumerate(zip(top_5['valor'].values, top_5['std'].values)):
            ax1.text(val + 0.5, i, f'{val:.2f} ± {std:.2f}',
                    va='center', fontsize=9, fontweight='bold')

        # Configurar
        ax1.set_yticks(y_pos)
        ax1.set_yticklabels(top_5['par'].values, fontsize=9, fontweight='bold')
        ax1.invert_yaxis()
        ax1.set_xlabel(metric_label, fontsize=11, fontweight='bold')
        ax1.grid(axis='x', alpha=0.3, linestyle='--')
        ax1.set_xlim(0, max_val_top + max_std_top + 8)

        # --- Subplot 2: BOTTOM 5 ---
        y_pos = np.arange(len(bottom_5))
        colors_bottom = plt.cm.Reds(np.linspace(0.3, 0.6, len(bottom_5)))

        ax2.barh(y_pos, bottom_5['valor'].values,
                color=colors_bottom,
                edgecolor='darkred',
                linewidth=1.2,
                alpha=0.8)

        # Calcular máximo para ajustar límites
        max_val_bottom = bottom_5['valor'].max()
        max_std_bottom = bottom_5['std'].max()

        # Valores en las barras
        for i, (val, std) in enumerate(zip(bottom_5['valor'].values, bottom_5['std'].values)):
            ax2.text(val + 0.5, i, f'{val:.2f} ± {std:.2f}',
                    va='center', fontsize=9, fontweight='bold')

        # Configurar
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(bottom_5['par'].values, fontsize=9, fontweight='bold')
        ax2.invert_yaxis()
        ax2.set_xlabel(metric_label, fontsize=11, fontweight='bold')
        ax2.grid(axis='x', alpha=0.3, linestyle='--')
        ax2.set_xlim(0, max_val_bottom + max_std_bottom + 8)

        plt.tight_layout()

        # Guardar
        output_path = os.path.join(output_folder, f'ranking_combinado_{metric}.png')
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
        plt.close()

        print(f"Ranking combinado generado: {output_path}")


def create_ranking_summary_table(results_dict, output_folder, dpi=300):
    """
    Crea una tabla resumen con los rankings para todas las métricas.

    Args:
        results_dict: Diccionario con DataFrames de comparación por modelo
        output_folder: Carpeta donde guardar la tabla
        dpi: Resolución de la imagen
    """

    os.makedirs(output_folder, exist_ok=True)

    # Configuración de estilo académico
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']

    # Métricas de test
    test_metrics = ['test_acc', 'test_bal_acc', 'test_f1_acc']
    metric_names = ['Exactitud', 'Exactitud Bal.', 'F1 Score']

    # Preparar datos para la tabla
    summary_data = []

    for metric, metric_name in zip(test_metrics, metric_names):
        # Recopilar pares
        all_pairs = []

        for model_name, df in results_dict.items():
            if metric in df.columns:
                for _, row in df.iterrows():
                    subset = row['sensor_subset']
                    value_str = str(row[metric])
                    mean_value = float(value_str.split('±')[0].strip())

                    all_pairs.append({
                        'par': f"{model_name} - {subset}",
                        'valor': mean_value
                    })

        # Ordenar y obtener mejor
        pairs_df = pd.DataFrame(all_pairs)
        pairs_df_sorted = pairs_df.sort_values('valor', ascending=False)

        best = pairs_df_sorted.iloc[0]
        worst = pairs_df_sorted.iloc[-1]

        summary_data.append({
            'Métrica': metric_name,
            'Mejor Configuración': best['par'],
            'Mejor Valor': f"{best['valor']:.2f}%",
            'Peor Configuración': worst['par'],
            'Peor Valor': f"{worst['valor']:.2f}%"
        })

    summary_df = pd.DataFrame(summary_data)

    # Crear visualización
    fig, ax = plt.subplots(figsize=(14, 4))
    ax.axis('tight')
    ax.axis('off')

    table = ax.table(
        cellText=summary_df.values,
        colLabels=summary_df.columns,
        cellLoc='left',
        loc='center',
        bbox=[0, 0, 1, 1]
    )

    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2.5)

    # Formato del encabezado
    for i in range(len(summary_df.columns)):
        cell = table[(0, i)]
        cell.set_facecolor('#34495E')
        cell.set_text_props(weight='bold', color='white')
        cell.set_edgecolor('white')
        cell.set_linewidth(1.5)

    # Formato de celdas
    for i in range(1, len(summary_df) + 1):
        for j in range(len(summary_df.columns)):
            cell = table[(i, j)]

            if i % 2 == 0:
                cell.set_facecolor('#ECF0F1')
            else:
                cell.set_facecolor('white')

            cell.set_edgecolor('#BDC3C7')
            cell.set_linewidth(0.5)

            # Columnas de mejor en verde, peor en rojo
            if j == 1 or j == 2:
                cell.set_text_props(color='darkgreen', weight='bold')
            elif j == 3 or j == 4:
                cell.set_text_props(color='darkred', weight='bold')

    plt.tight_layout()

    output_path = os.path.join(output_folder, 'ranking_resumen_tabla.png')
    plt.savefig(output_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"Tabla resumen de rankings generada: {output_path}")


# ============================================================================
# EJECUCIÓN - RANKINGS
# ============================================================================

timestamp = '2025-12-08_00-27-50'
date_str = '2025-12-08'

FINAL_BASE_FOLDER = os.path.join(
    '/content/drive/My Drive/TFM_2/resultados_entrenamiento/',
    date_str,
    'Fase_1',
    timestamp,
    'resultados_entrenamiento'
)

# Crear carpeta específica para rankings
ranking_output = os.path.join(FINAL_BASE_FOLDER, 'rankings')

print("\n" + "="*80)
print("GENERANDO RANKINGS DE CONFIGURACIONES")
print("="*80 + "\n")

# Rankings individuales (top 5 y bottom 5 separados)
create_top_bottom_rankings(results, ranking_output, dpi=300)

# Rankings combinados (top 5 y bottom 5 en una sola imagen)
create_combined_top_bottom_ranking(results, ranking_output, dpi=300)

# Tabla resumen de rankings
create_ranking_summary_table(results, ranking_output, dpi=300)

print("\n" + "#"*80)
print("# RANKINGS COMPLETADOS")
print(f"# Archivos guardados en: {ranking_output}")
print("#"*80 + "\n")


GENERANDO RANKINGS DE CONFIGURACIONES

Ranking Top 5 generado: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/resultados_entrenamiento/rankings/ranking_top5_test_acc.png
Ranking Bottom 5 generado: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/resultados_entrenamiento/rankings/ranking_bottom5_test_acc.png
Ranking Top 5 generado: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/resultados_entrenamiento/rankings/ranking_top5_test_bal_acc.png
Ranking Bottom 5 generado: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/resultados_entrenamiento/rankings/ranking_bottom5_test_bal_acc.png
Ranking Top 5 generado: /content/drive/My Drive/TFM_2/resultados_entrenamiento/2025-12-08/Fase_1/2025-12-08_00-27-50/resultados_entrenamiento/rankings/ranking_top5_test_f1_acc.png
Ranking Bottom 5 generado: /content/drive/My Drive/T